In [1]:
import os
import sys as sys

import itertools
from itertools import cycle,groupby
import re
from re import search

import regex
import xlrd
#import xlwt
# To install run: !conda install xlwt or !pip install xlwt or similar in your environment

import glob
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import seaborn
import numpy as np
from numpy import linspace
from copy import deepcopy
import traceback
import warnings
import argparse
import subprocess
import ftplib as ftp

#Don't need:
# import scipy.stats
# import scipy.cluster
# from scipy import interp
# from scipy.stats import norm
# from scipy.stats.kde import gaussian_kde
# import sklearn

import ete3 as ete3
from ete3 import Tree, PhyloTree #, TreeStyle, TextFace, add_face_to_node #Looks like these are deprecated??

import Bio as Bio
from Bio import SeqIO, AlignIO
from Bio import Phylo, _utils
from Bio.Phylo import BaseTree

#!conda install IO

In [2]:
!pwd

/nobackup1c/users/payette/PVCtree


In [3]:
os.chdir('/nobackup1c/users/payette/PVCtree/sequences')
!pwd

/nobackup1c/users/payette/PVCtree/sequences


In [4]:
def fasta_iter(fasta_name):
    fasta_nam = open(fasta_name)
    fasta_s = (x[1] for x in groupby(fasta_nam, lambda line: line[0] == ">"))
    for header in fasta_s:
                                      # drop the ">" #turned off
        headerStr = header.__next__() #[1:].strip()
        # join all sequence lines to one.
        seq = "".join(s.strip() for s in fasta_s.__next__())
        yield (headerStr, seq)

In [5]:
def fast_fasta(fasta_file,protein_name):
    for header, seq in fasta_iter(fasta_file):
        #print(header)
        if header.find(protein_name) != -1:
            print(header,seq)
            #print(seq)
            return(header,seq)
        else:
            continue  

In [6]:
class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [7]:
def checkGenome(file_index):    
    num_complete = 0    
    for protein_name in large_subunits+small_subunits:
        num_complete = 0
        print(protein_name)
        for header, seq in fasta_iter(file_index):
            if header.find(protein_name) != -1:
                print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                continue
            print(num_complete)

In [8]:
#Manually import data for protein identification from .faa RefSeq / Genbank files
#Please note this is not a automatic process and needs by-hand adjustment

large_subunits=['50S ribosomal protein L1 [','50S ribosomal protein L2 [','50S ribosomal protein L3 [',
                '50S ribosomal protein L4 [','50S ribosomal protein L5 [','50S ribosomal protein L6 [',
                '50S ribosomal protein L10 [','50S ribosomal protein L13 [','50S ribosomal protein L14 [',
                '50S ribosomal protein L15 [','50S ribosomal protein L18 [','50S ribosomal protein L22 [',
                 '50S ribosomal protein L23 [','50S ribosomal protein L24 [','50S ribosomal protein L29 [']
print(len(large_subunits))

15


In [9]:
small_subunits=['30S ribosomal protein S2 [','30S ribosomal protein S3 [','30S ribosomal protein S4 [',
                '30S ribosomal protein S5 [','30S ribosomal protein S7 [','30S ribosomal protein S8 [','30S ribosomal protein S9 [',
                '30S ribosomal protein S10 [',
 '30S ribosomal protein S11 [','30S ribosomal protein S12 [','30S ribosomal protein S13 [','30S ribosomal protein S14 [',
 '30S ribosomal protein S15 [','30S ribosomal protein S17 [','30S ribosomal protein S19 [']
print(len(small_subunits))

#Notes: S12 needs fixing and regenerating

15


In [10]:
#TO DO MAKE A PROTEIN DF so I can call name and string search easily.
proteins = ['L1','L2','L3','L4','L5','L6',
            'L10','L13','L14','L15','L18','L22',
            'L23','L24','L29','S2','S3','S4',
            'S5','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S17','S19']

In [11]:
# In riboDB:
# ul1;ul2;ul3;ul4;ul5;ul6;ul10;ul13;ul14;ul15;ul18;ul22;ul23;ul24;ul29;
# us2;us3;us4;us5;us7;us8;us9;us10;us11;us12;us13;us14;us15;us17;us19;

In [12]:
!pwd

/nobackup1c/users/payette/PVCtree/sequences


In [13]:
# Commented out code blocks for checking genome files.

In [14]:
#Manually upload zipped genomes to directory -- check
#Check the number of .gz genome files using the command line below.
#Checksum: should be == XXX

#!ls -ltr *.gz | wc

In [15]:
#Checksum: Should print for 106 genomes

!ls -lR *.faa | wc -l

106


In [16]:
# Upload Excel file to create Pandas Dataframe of Taxa
taxa_df = pd.read_csv('PVC_group_Rprots.csv') #Checksum: On 10-6-20 there should be 227
#7-6-21 plus 80 (-1 duplicate) = 306

In [17]:
#Re-index Pandas Dataframe to use the .faa filename for the index, used in loops
taxa_df.index=taxa_df.Filename 

In [18]:
#Print length of taxa dataframe and print last 5 records.
print(len(taxa_df))
taxa_df.tail()

106


,Index,Name,URL,Filename,FullURL,Accession,TaxID
Filename,,,,,,,
GCA_012799725.1_ASM1279972v1_protein.faa,101,Oligosphaeraceae bacterium,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_012799725.1_ASM1279972v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_012799725.1 ASM1279972v1,2699751
GCA_018648945.1_ASM1864894v1_protein.faa,102,Victivallales bacterium,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_018648945.1_ASM1864894v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_018648945.1 ASM1864894v1,2212475
GCA_001803205.1_ASM180320v1_protein.faa,103,Lentisphaerae bacterium GWF2_38_69,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803205.1_ASM180320v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803205.1 ASM180320v1,1798570
GCA_003635265.1_ASM363526v1_protein.faa,104,Candidatus Poribacteria bacterium,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_003635265.1_ASM363526v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_003635265.1 ASM363526v1,2026781
GCA_001803235.1_ASM180323v1_protein.faa,105,Lentisphaerae bacterium RIFOXYA12_64_32,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803235.1_ASM180323v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,GCA_001803235.1 ASM180323v1,1798577


In [19]:
#For riboDB -- print out taxa list
for index,row in taxa_df.iterrows():
    print('#',row.Name.replace(' ', '_'))

# Candidatus_Brocadia_sp.
# Candidatus_Brocadiae_bacterium
# Phycisphaera_mikurensis_NBRC_102666
# Sedimentisphaerales_bacterium
# Humisphaera_borealis
# Gimesia_maris
# Gemmata_obscuriglobus_UQM_2246
# Isosphaera_pallida_ATCC_43644
# Rhodopirellula_baltica_SH_1
# Rubinisphaera_brasiliensis_DSM_5305
# Planctopirus_limnophila_DSM_3776
# Schlesneria_paludicola_DSM_18645
# Polystyrenella_longa
# Alienimonas_californiensis
# Calycomorphotria_hydatis
# Caulifigura_coniformis
# Crateriforma_conspicua
# Fuerstia_marisgermanicae
# Thalassoglobus_polymorphus
# Symmachiella_dynata
# Thermogutta_terrifontis
# Paludisphaera_borealis
# Singulisphaera_acidiphila_DSM_18658
# Tautonia_marina
# Posidoniimonas_polymericola
# Bythopirellula_goksoyri
# Blastopirellula_marina_DSM_3645
# Blastopirellula_cremea
# Blastopirellula_retiformator
# Roseimaritima_sp._JC651
# Roseimaritima_multifibrata
# Roseimaritima_ulvae
# Stieleria_maiorica
# Stieleria_varia
# Stieleria_neptunia
# Candidatus_Laterigemmans_bacul

In [20]:
# Commented out -- for genome pulling.
#Code below will unzip automatically from a directory
#for index,row in taxa_df.iterrows():
#    subprocess.call(['gunzip', '%s/%s.gz' % ('/nobackup1c/users/payette/PVCtree/sequences',(index))])

In [21]:
#Should print 106 for PVC tree
!ls -lR *.faa | wc -l

106


In [22]:
import os
Sequence_list=list()
directory = os.fsencode('/nobackup1c/users/payette/PVCtree/sequences')    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".faa"):
        print(filename)
        Sequence_list.append(filename)

GCF_000165715.2_ASM16571v3_protein.faa
GCF_008035925.1_ASM803592v1_protein.faa
GCF_007753095.1_ASM775309v1_protein.faa
GCF_902806995.1_15-2067_O50_protein.faa
GCF_007741475.1_ASM774147v1_protein.faa
GCF_000196115.1_ASM19611v1_protein.faa
GCF_000470765.1_ASM47076v1_protein.faa
GCF_014230145.1_ASM1423014v1_protein.faa
GCA_003635265.1_ASM363526v1_protein.faa
GCF_007559335.1_ASM755933v1_protein.faa
GCF_013618625.1_ASM1361862v1_protein.faa
GCF_007860045.1_ASM786004v1_protein.faa
GCF_007475525.1_ASM747552v1_protein.faa
GCF_007747795.1_ASM774779v1_protein.faa
GCF_013036045.1_ASM1303604v1_protein.faa
GCF_902807005.1_14-2711_R47_protein.faa
GCF_009731575.1_ASM973157v1_protein.faa
GCF_007744255.1_ASM774425v1_protein.faa
GCF_007860075.1_ASM786007v1_protein.faa
GCF_007859935.1_ASM785993v1_protein.faa
GCF_007748115.1_ASM774811v1_protein.faa
GCF_003096415.1_ASM309641v1_protein.faa
GCF_000297415.1_ASM29741v1_protein.faa
GCF_001613545.1_ASM161354v1_protein.faa
GCF_003096195.1_ASM309619v1_protein.faa
G

In [23]:
#WARNING NAMES ARE CASE SENSITIVE!!!

In [24]:
os.chdir('/nobackup1c/users/payette/PVCtree/sequences')

In [25]:
### Checking Routine below

In [26]:
for index,row in taxa_df[:].iterrows():
    checkGenome(index)

50S ribosomal protein L1 [
>QQR67113.1 50S ribosomal protein L1 [Candidatus Brocadia sp.]
 MSKRSKRYVESKKIVDSEKKYELKEAVKLLKSFKRTKFDASVEVAMKLGIDPKQSDQLIRGSVSLPKGIGRSLKVIVFAAGEKAALAKNAGADEVGAEELVKKVEGGWADFDVAIATSDMMRFVGKLGRVLGPQGKMPSPKSGTVTDDIGTAVKEFKAGKIEYRTDAGGNVHSIVGKVSFSEKDLEDNINAFVKQIVNSRPASAKGIFIEKVSVSSTMSPGIMIEV
1
50S ribosomal protein L2 [
>QQR67124.1 50S ribosomal protein L2 [Candidatus Brocadia sp.]
 MGIIYYKPVTAGRRFASVSDFSEITKARPEKSLLEPKKKTGGRNAEGKITAAHIGGGSRRHYRIIDFKRKKDNIPAKVAGIEYDPNRSARIALLHYSDGEKRYILAPDSLRVGQTLMSGEKVEPEVGNCMPLKNIPLGLEIHNIEMRVGQGGQLVRSAGGTAKLMAKEGNYAHIVLPSGEVRKVFYGCRATIGKIGNVEHLNIRLGKAGRKRWKGRRPHVRGVAQNPVSHPLGGGEGRSGGGRHPCSKTGLLAKGGKTRKKKALSNKFIIRRRRIGARGNL
1
50S ribosomal protein L3 [
>QQR67121.1 50S ribosomal protein L3 [Candidatus Brocadia sp.]
 MLHGFLGKKIGMTQIYTDEGKLLPVTLIQAGPCSVIQIKTFENDGYSAIQIGFDDKKKKSATRPEIGHVSKLSLEPKRFIREIKPEMGAELKIGQSITVDIFQDVKMVDVIGVTKGKGYAGVMKRWNFKGGLNTHGSTRHRPPGSIGSNTDPGRVMRGKKMAGRLGGERVTIKNIDVVKIDANRSLLFLKGAVPGHKGSYLILNKARVEKVGK


50S ribosomal protein L15 [
>NUM34068.1 50S ribosomal protein L15 [Candidatus Brocadiae bacterium]
 MNLFEIKTAPIPKKVSKRVGRGSGSGKGTSCGRGLRGQKKRQGNTDLVVFEGGIMPLFRRLPKRGFQHDRFDDLWVTVNVEDLNCFQDGEEVNLESVTQKGIVKFAKSKKTARLKILGKGELTVQNLKIKAYKVSEKAKEQLEKAKATLEILKIMKKVKKSRRRKSS
1
50S ribosomal protein L18 [
>NUM34066.1 50S ribosomal protein L18 [Candidatus Brocadiae bacterium]
 MDSTLKDMRRWRRHCHIRKRLIGTPERPRLGVFRSSKHIYCQIIDDTFVDKNAKCSGRTLLACSSLTPTIAKQLTNRNGGNIKTAELIGTEIAKMAIEKGIKAVAFDRGGYKYHGRVKALAEAARKAGLQF
1
50S ribosomal protein L22 [
>NUM34055.1 50S ribosomal protein L22 [Candidatus Brocadiae bacterium]
 MKFYAIHKYARITPRKARYVVDMIRNKTANEALEILATVPKRASYFVKKVLNSAVANAAQQEGIKLGNLFVREAKVHGGPTLKRYRPGPMGRAMGILKHTSHIEITLVEKAPEVEKPKDKAAKAKKDQKAETKKDKRGE
1
50S ribosomal protein L23 [
>NUM34052.1 50S ribosomal protein L23 [Candidatus Brocadiae bacterium]
 MARDAYSIILRPIMTEKSNIDVELQNAYHFEVALDANKVEIKKALEKIYAHKGIKVARVNIMNKHPKKRRFRFRQGYTKAWKKAIVFLDKDYKLEMF
1
50S ribosomal protein L24 [
>NUM34061.1 50S ribosomal 

30S ribosomal protein S5 [
>WP_014438268.1 30S ribosomal protein S5 [Phycisphaera mikurensis]
 MAETLDGNQLESTTVAINRTSTTVKGGRRLSFSALVVVGNRDGKVGIGYGKGRGVPVGIEKAQKVARKEMVAIKRLGATVPHEVIGRSCASTVKLIPAAPGTGVIAGGTVRAVLEMAGVRDCLTKAYGSTNKINLCRAVMDALDQLRTREEIAKLRGVEIEKSTVDEALEANKRYMTEEGDAGPKKKAKGPTNQKDEDAKKTRGRGGRGGPGGGRGGNAGGPAAPKPDADKDAQTADGEGNVATGIAGSVAGAAGAVASVAGGAVGAVLGAVTGGSDTAGDKPETTDGDDVGAAAAPATPDAADVPRDAETEEGKA
1
30S ribosomal protein S7 [
>WP_014438029.1 30S ribosomal protein S7 [Phycisphaera mikurensis]
 MGRGFTASETQLKPDPRHGDLVLSRFINCIMKDGKKTRAQRVVYGAFDLIDERLKEDAEKEAEAKKKAAEDKAAGKDVKPTNAQLTGTPEEEEGERSAITLFHEALENVRPNVEVRSRRVGGTNYQVPTQVSKKRAQSLAFRWIIDAVRSEKGRPTAQKLARELWDASHREGRAMSKRDQTHRMADANKAFAHFAW
1
30S ribosomal protein S8 [
>WP_014438271.1 30S ribosomal protein S8 [Phycisphaera mikurensis]
 MALSDPIADMLTRIRNASANGAERVDCRNSRVCRGIAAALEHEGYITSSEVLAGEQFGGLGVLRVTLKYGPFGEKLFHKLKRASKPGCRVYTKVKDLPRPLAGLGVAILTTPVGVLSDRQCREKNVGGELICTVE
1
30S ribosomal protein S9 [
>WP_041378610.1 30S ribosomal pr

>MBN2132643.1 30S ribosomal protein S10 [Sedimentisphaerales bacterium]
 MAAETEKIRIRMEAYDHRTLDASAKEIVEHARRTNARVSGPIPLPTRIERYTVLRSPHVDKKSREQFELRTHKRLIDIHEANARTAEVLNRLVVPAGVFVKIKA
1
30S ribosomal protein S11 [
>MBN2132669.1 30S ribosomal protein S11 [Sedimentisphaerales bacterium]
 MAKGTKRRVRRNVVRAIVNIKATFNNTVITITDMDGDTICASSAGCVGFKGSRKSTPFAAQQAAQSCANVAMRNGVREVEIKVKGPGSGRESAISAMQAAGLRITSIEDVTPLPHNGCRPPKRRRV
1
30S ribosomal protein S12 [
>MBN2132639.1 30S ribosomal protein S12 [Sedimentisphaerales bacterium]
 MPTINQLVRKPRAKSKGHKRVSDISGSPQKRGVCLIVRTQTPKKPNSALRKVARVRLTNGKEVTAYIPGIDHNLQEHSTVIIRGGRVRDLPGVRYHIVRGVLDCAGVSGRQQGRSKYGAKKGK
1
30S ribosomal protein S13 [
>MBN2132668.1 30S ribosomal protein S13 [Sedimentisphaerales bacterium]
 MPRIVGIDIPDNKSIWISLTYIPGIGRYTSEQIIKEARIEQMTKAGKLTEDELSRITQIIDRNYIVGGQLRRQVAQNIARLRDIGCYRGIRHRRGLPVRGQCTQSNARTRKGPRKTVAGKKGVKEKR
1
30S ribosomal protein S14 [
>MBN2132657.1 type Z 30S ribosomal protein S14 [Sedimentisphaerales bacterium]
 MSTKALENKAKKKQKFRVREYTRC

50S ribosomal protein L6 [
>WP_154901405.1 50S ribosomal protein L6 [Gimesia maris]
 MSRIGKKPVPVAAGVEVKVDDTSISVKGKHGDLSFAFHPSMTVELDSETNEINVIRPDDSRQSRALHGLTRALIANMVQGVETPFVRKLEIQGVGYQASLNGNKLSLQVGFANTIVLEVPAGVICELPSSTNIVLTSADKHAVGQFAANIRSVRPPEPYKGKGIRYEGEYVRRKAGKAFAN
1
50S ribosomal protein L10 [
>WP_154901408.1 50S ribosomal protein L10 [Gimesia maris]
 MSKIVKEMIISEIEARISEVRDFVVVDSAKVDAITDNGFRLKLQEKGIASLTVKNSLARRALANIGVEGLDDVLKGPSTLVWGGEDIVALSKEMTKWASDISTLEIKGGVTEGTPLSSEDVTKLSKSPGRMELIGEIVSRILGPGAQLAGAIKGPGGTLAGQIKTISEGEAA
1
50S ribosomal protein L13 [
>WP_230682375.1 50S ribosomal protein L13 [Gimesia maris]
 MSTTKTFMANAKTINPEWFVVDADNLIVGRLAAKIATVLMGKHKPTYTPHVDTGDYVIVVNCDKVKFSGKELAHDSHPYFSQKMLRKGYATYSGYPSGLKVVTAEKKLERGQSTQVLSEAVRRMLPKNKLGRQMLKKLKLYAGPTHENQSQQPQDMPAHLLP
1
50S ribosomal protein L14 [
>WP_002649723.1 MULTISPECIES: 50S ribosomal protein L14 [Gimesia]
 MIQMQTDLDVCDNSGAKVARCIKVLGGTGRRTAEVGDVIVVSIQKTLAGSNIKKGQVLRGVIVRTRYPCRRDDGSYVRFDRNAMVLIDGEGNPRGTRIFGAVARELRERKYMKIISLANEVV

...

30S ribosomal protein S15 [
>WP_007281373.1 30S ribosomal protein S15 [Lentisphaera araneosa]
 MTKEKRQEIIAAFQRKEGDVGSPEVQIALLTGRIIELTEHMKKNKKDYSTSRGLITMVNNRKKLLRYLNSENHAGYIEITDKLKIRR
1
30S ribosomal protein S17 [
>WP_007280391.1 30S ribosomal protein S17 [Lentisphaera araneosa]
 MSENTNQERGNRKQRVGIVTSDKMDKTIVVTVNRKARHAKFKKVINISKKYYAHDEKNEAKIGDQVVIAETRPTSKKKCWRLMSIQSTAE
1
30S ribosomal protein S19 [
>WP_007279820.1 30S ribosomal protein S19 [Lentisphaera araneosa]
 MPRSIKKGPFVDAHLAAKVEKATASGDHKAIKTWSRRSMILPTFVGLNFEVHNGKEFIPVFVTDNMVGHKLGEFALTRAFRTHGVMSGK
1
50S ribosomal protein L1 [
>QSH39999.1 50S ribosomal protein L1 [Lentisphaerae bacterium]
 MHRSKLYSSQMQCFSRQQNYSVTEAIEIIKKMPHIKFDETVDVAFKLGIDPRKSDQNVRGACPLPKGTGKKISVCVIASGPQADEAKASGADFVGFEDLIEKIKGGWLDFDILIATPDAMKQVRPLGRVLGPRGLMPNPKTGTVTNDIAAAVNEAKAGRVEFRADRGACVHVPLGKLSFANEDLLENFDAVFDALQKAKPATAKGTYFVSCTLSSTMSPGVKVDVKEIVRQKL
1
50S ribosomal protein L2 [
>QSH39988.1 50S ribosomal protein L2 [Lentisphaerae bacterium]
 MAIKSYNPTTPSRRNMTVSDFA

>WP_176012655.1 50S ribosomal protein L5 [Victivallis sp. Marseille-Q1083]
 MPDMSKKYRDEVRPALQAQLGLKNVMQIPKLEKIVISTGMKSSAERDAFSEARKHLAAMTGQMPVITKARKNVSNFKLRVGMQIGVMVTLRGARMYEFLDRFVHNALPRVRDFRGIPSSGFDGRGNYNVGIQDISVFTEVDPDKLKYPIGVNITMVTTAENDDEARALLKLLEVPFAE
1
50S ribosomal protein L6 [
>WP_176012652.1 50S ribosomal protein L6 [Victivallis sp. Marseille-Q1083]
 MSRIGKKPIPVPAGVTVTIADGTVTVAGKEKLQLALPPKVAVEMKDGHVLVSQVEDTRTSSAMQGLARSLINNMVIGVSQGFKKELQIIGVGYKAQISGSKLVLNLGYSHVINYDVPAGVKVTVADNVKITVEGADKQLVGETAATIRRFRKPEPYKGKGIRYVDEQITLKEGKSVG
1
50S ribosomal protein L10 [
>WP_176013154.1 50S ribosomal protein L10 [Victivallis sp. Marseille-Q1083]
 MRSEKSHLVNYIGGLLTGSDYIYFVSYQGLQVKQFSQLRNQLAGLGAECHVLKNTLIRKAAELSSLEALAKSELTDSTAVICGKGDPSAVAKAVIEFGKTNEALAAKGGYFEGALLSVADFDAIASLPPKEVLQAQLLGVLQAPSRNLVSLLNAKAASILNVLNAYKEKLDN
1
50S ribosomal protein L13 [
>WP_176012742.1 50S ribosomal protein L13 [Victivallis sp. Marseille-Q1083]
 MKTYLAKTGEINREYVLFDASEAPVGRLAVMIANALRGKDKPTYTPHIDTGAFVVVVNADNAVFTGRKSEEKIYV

>NMA43993.1 30S ribosomal protein S13 [Oligosphaeraceae bacterium]
 MPRILGVDIPNNKHVNVSLTYIYGIGPSVANEICRRAQIDPMLKAGDLSESNIAAILQILQESFLVEGDLRRQMAQNIRRLIAIGSYRGLRHRRNLPVRGQRTRTNARTRKGSKKTIGAIRDKSARKQAGK
1
30S ribosomal protein S14 [
>NMA42579.1 type Z 30S ribosomal protein S14 [Oligosphaeraceae bacterium]
 MAKKSLIVKSQKPAKFSARAYTRCQRCGRPRAYIRKFHLCRICFRELANSGMIPGVTKASW
1
30S ribosomal protein S15 [
>NMA43544.1 30S ribosomal protein S15 [Oligosphaeraceae bacterium]
 MKKEQKEATIKEFQRSPNDVGSVEVQVALATGRIVELTEHMKIHKKDFSTRRGLVRQVNHRRNLLNYLQKRDYERYITLIRKLGLRR
1
30S ribosomal protein S17 [
>NMA42583.1 30S ribosomal protein S17 [Oligosphaeraceae bacterium]
 MSAAEENKGAAPVRNLRKVLNGTVVSDKMDKTLVVQVDRRTQHRLYKKVIKVTKKYYVHDEKNEARAGDQVKIMEVRPLSKSKRWRLVSITQRAATE
1
30S ribosomal protein S19 [
>NMA42742.1 30S ribosomal protein S19 [Oligosphaeraceae bacterium]
 MARSIKKGFFVDDSLMKKVDAMNSRNEKRVVKTWSRRSMITPDFVGHTFSVHNGKTFFNVFVTENMVGHRLGEFAPTRNFRGHGAISGK
1
50S ribosomal protein L1 [
50S ribosomal protein L2 [
50S rib

30S ribosomal protein S13 [
>OGV19898.1 30S ribosomal protein S13 [Lentisphaerae bacterium GWF2_38_69]
 MPRILGVDIPGKKRIEISLRYLYGIGPAKALEIVKKAKIPLGTKADDLTADQVSSITTMLQNDYLVEGDLRRAIMQDIRRLTAIGSYRGVRHRRGLPVRGQRTKTNARTRKGRAKTVGAIRDKTQRKLANKDE
1
30S ribosomal protein S14 [
>OGV18985.1 30S ribosomal protein S14 [Lentisphaerae bacterium GWF2_38_69]
 MAKKSLVVKSEREPKFKVRKYNRCKKCGRSRAYIRKYQLCRICFRELASSGQIPGVIKASW
1
30S ribosomal protein S15 [
>OGV21662.1 30S ribosomal protein S15 [Lentisphaerae bacterium GWF2_38_69]
 MKSDAKKQTVDKFALSKNDTGSAPVQVASLSTEIKSLTEHLKEHKKDHSSRRGLIAKVNKRRKLLNYFKREKPEECEKLIQDLKIRHQ
1
30S ribosomal protein S17 [
>OGV18981.1 30S ribosomal protein S17 [Lentisphaerae bacterium GWF2_38_69]
 MENEIKDTRKNFRKEREGVVLSDCQNKTIVVEVTRRTSHPLYKKVVKSSKKYYAHDEENKAKKGDKVTITETRPLSKLKRWRLVKVHGKEQQS
1
30S ribosomal protein S19 [
>OGV19953.1 30S ribosomal protein S19 [Lentisphaerae bacterium GWF2_38_69]
 MARSVKKGPFVDSHLLDKVEKMNKSSAKKPIKTWSRRSVILPEFVGLTFTVHNGKTFISVFITENMIGHRLGEFALTRTFRSHGVVSGK
1
5

>OGV72281.1 50S ribosomal protein L4 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MSTTLPIFDKSGSQIGGLDLSPAWLEREKGDQAVHDSVVACLAGMRAGTASTKTRGKVRGGGAKPWRQKGTGRARAGSNRSPVWRGGGIIFGPIPRSFAKHVNRKVEKLAMRRVFTERLDENAVIVVDDVVLDTPKTKGLTALLKTLGAGVDALVLVDKVEDNVRLAAQNLPSVRLMTAVTVNPYWMLLYKKIVVTRAALEVLGGRIGAREKAE
1
50S ribosomal protein L5 [
>OGV70004.1 50S ribosomal protein L5 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MKSVFYDYYLNDVVPKLMQKRAYKNRLEVPRLTKIVVNSGVGTAKDREVITEAVSTLGIITGQRPVVTKSRKSISNFKLREGMSVGVCVTLRGGAMYNFVYRLINVALPRVRDFRGVPASAFDGAGNYSMGLPDQSVFTEIELDKVKHTIGMNISIVTTARTDAEARDLLALLGVPFAS
1
50S ribosomal protein L6 [
>OGV70007.1 50S ribosomal protein L6 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MSRIGKKPVVLADKVKVEVDGSTVTVQGPKGSLSYQMAPGLEVRVEAKNLFVENVADAKTLNAVHGLTRSLINNMVVGVTVGYREQLEVQGVGYRGQCKGRHLTLSVGYSSPVEYDVPEGVILTMPNQTTIVVEGIDKHLVGEAAATIRRFRPPDAYKGKGVRYAGEQVTLKEGKTVS
1
50S ribosomal protein L10 [
>OGV72265.1 50S ribosomal protein L10 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MVKDLQALVGPATCLFL

In [27]:
#Sub routine which prints the number of completed genomes, based on index file of Pandas dataframe, for each protein.
protein_name='50S ribosomal protein L1 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df.iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            #print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L1 [
GCA_016699285.1_ASM1669928v1_protein.faa
GCA_013359215.1_ASM1335921v1_protein.faa
GCF_000284115.1_ASM28411v1_protein.faa
GCA_016932895.1_ASM1693289v1_protein.faa
GCF_015169395.1_ASM1516939v1_protein.faa
GCF_007747015.1_ASM774701v1_protein.faa
GCF_901538385.1_ASM90153838v1_protein.faa
GCF_000186345.1_ASM18634v1_protein.faa
GCF_000196115.1_ASM19611v1_protein.faa
GCF_000165715.2_ASM16571v3_protein.faa
GCF_000092105.1_ASM9210v1_protein.faa
GCF_000255655.1_ASM25565v1_protein.faa
GCF_007750395.1_ASM775039v1_protein.faa
GCF_007743815.1_ASM774381v1_protein.faa
GCF_007745435.1_ASM774543v1_protein.faa
GCF_007745175.1_ASM774517v1_protein.faa
GCF_007752935.1_ASM775293v1_protein.faa
GCF_001983935.1_ASM198393v1_protein.faa
GCF_007744255.1_ASM774425v1_protein.faa
GCF_007747995.1_ASM774799v1_protein.faa
GCF_002277955.1_ASM227795v1_protein.faa
GCF_001956985.1_ASM195698v1_protein.faa
GCF_000242455.2_ASM24245v3_protein.faa
GCF_009177065.1_ASM917706v1_protein.faa
GCF_007859935.1

In [28]:
#reminder how many taxa we have
len(taxa_df)

106

In [29]:
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name)
    for index,row in taxa_df.iterrows():
        #print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                #print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                #print('ERROR!',index,protein_name)
                continue
    print(num_complete)

50S ribosomal protein L1 [
104
50S ribosomal protein L2 [
103
50S ribosomal protein L3 [
103
50S ribosomal protein L4 [
103
50S ribosomal protein L5 [
105
50S ribosomal protein L6 [
105
50S ribosomal protein L10 [
105
50S ribosomal protein L13 [
104
50S ribosomal protein L14 [
105
50S ribosomal protein L15 [
103
50S ribosomal protein L18 [
104
50S ribosomal protein L22 [
103
50S ribosomal protein L23 [
104
50S ribosomal protein L24 [
101
50S ribosomal protein L29 [
104
30S ribosomal protein S2 [
102
30S ribosomal protein S3 [
104
30S ribosomal protein S4 [
105
30S ribosomal protein S5 [
104
30S ribosomal protein S7 [
104
30S ribosomal protein S8 [
106
30S ribosomal protein S9 [
103
30S ribosomal protein S10 [
104
30S ribosomal protein S11 [
102
30S ribosomal protein S12 [
105
30S ribosomal protein S13 [
105
30S ribosomal protein S14 [
103
30S ribosomal protein S15 [
104
30S ribosomal protein S17 [
93
30S ribosomal protein S19 [
104


In [30]:
### Checking Routine

In [31]:
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name)
    for index,row in taxa_df.iterrows():
        #print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                #print(header,seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                #print('ERROR!',index,protein_name)
                continue
    print(num_complete)

50S ribosomal protein L1 [
104
50S ribosomal protein L2 [
103
50S ribosomal protein L3 [
103
50S ribosomal protein L4 [
103
50S ribosomal protein L5 [
105
50S ribosomal protein L6 [
105
50S ribosomal protein L10 [
105
50S ribosomal protein L13 [
104
50S ribosomal protein L14 [
105
50S ribosomal protein L15 [
103
50S ribosomal protein L18 [
104
50S ribosomal protein L22 [
103
50S ribosomal protein L23 [
104
50S ribosomal protein L24 [
101
50S ribosomal protein L29 [
104
30S ribosomal protein S2 [
102
30S ribosomal protein S3 [
104
30S ribosomal protein S4 [
105
30S ribosomal protein S5 [
104
30S ribosomal protein S7 [
104
30S ribosomal protein S8 [
106
30S ribosomal protein S9 [
103
30S ribosomal protein S10 [
104
30S ribosomal protein S11 [
102
30S ribosomal protein S12 [
105
30S ribosomal protein S13 [
105
30S ribosomal protein S14 [
103
30S ribosomal protein S15 [
104
30S ribosomal protein S17 [
93
30S ribosomal protein S19 [
104


In [32]:
#All in one -- one shot, iterate over genome list, grab fasta, rewrite, print to screen
#Manually build files.
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!
num_complete = 0    
for protein_name in large_subunits+small_subunits:
    num_complete = 0
    print(protein_name,'### BREAK ###')
    for index,row in taxa_df.iterrows():  #[taxa_df.Name=='Desulfurella acetivorans A63']
        print(index) #Turn this on to get specific.
        index_complete = 0
        for header, seq in fasta_iter(index):
            index_complete = index_complete+1
            if header.find(protein_name) != -1:
                
                new_header = taxa_df.loc[index][1]
                new_header =  new_header.replace(' ', '_')
                
                print('>'+new_header+'\n'+seq) #Turn this on to get specific.
                num_complete = num_complete+1
            else:
                continue
                print('ERROR!',index,protein_name)
    #print(num_complete)

50S ribosomal protein L1 [ ### BREAK ###
GCA_016699285.1_ASM1669928v1_protein.faa
>Candidatus_Brocadia_sp.
MSKRSKRYVESKKIVDSEKKYELKEAVKLLKSFKRTKFDASVEVAMKLGIDPKQSDQLIRGSVSLPKGIGRSLKVIVFAAGEKAALAKNAGADEVGAEELVKKVEGGWADFDVAIATSDMMRFVGKLGRVLGPQGKMPSPKSGTVTDDIGTAVKEFKAGKIEYRTDAGGNVHSIVGKVSFSEKDLEDNINAFVKQIVNSRPASAKGIFIEKVSVSSTMSPGIMIEV
GCA_013359215.1_ASM1335921v1_protein.faa
>Candidatus_Brocadiae_bacterium
MSDTTEKIEQKSHKRRRPGPPNRRQPSKRHIENKSKVDKIKKYSLEEAVNLLKSTKNAKMDETVDIVMKLGIDPRKSEQMVRGSVSLPKGIGKQLKVIAFVEGDLAEEAKKAGADLVGAGDLVDKILNENWMDFDVAIAHPGMMRFVGKLGKVLGPKGKMPSPKSGTVTPDVVKAVTEFKAGKVEYRADSFGNVHAPVGKKSFPPENLIANIDTFIEHIKQVKPPTAKGTYIQKVVISSTMGPGVPIELKR
GCF_000284115.1_ASM28411v1_protein.faa
>Phycisphaera_mikurensis_NBRC_102666
MAGKRYRSDLEKATTDPVPLIEAVNRVKAMKKLKFDQTINLCVHLGIDPKQADQQLRGAISLPNGIGGDPKRVVAFVAPEKEEAAKEAGAIRAGSEDLVKEIEGGWQAFDVAIAEPAMMRVVARLGKVLGPRGLMPSPKAGTVTPNVAQAVKEFVAGKVEYRNDDGGNIHVVIGRQSFEPGQLAENAQAMIDHLSGRKPSSSKGVYFKKATIAGTMTPGVAVQLAAAAD
GCA_016932895.1_ASM1693289v1_protei

GCF_007859755.1_ASM785975v1_protein.faa
>Blastopirellula_retiformator
MVQHSKRYRALAEKTSEEVMPLDKAVELLKTFNTTKFDQTVDCAMRLGIDPKQADQLVRGAIVLPHGIGKELRVVVFAKGDNVRLAEESGADAVGAEDLAKKIKDGWLDFDACIATPDMMGLVGPLGRVLGPRGLMPSPRAGTVTPDVARVVKEYKAGKVEFRNDATGIVHAVVGKMSFDEAKLVDNIRAFMDQILSMKPAAAKGIYVKSVAIAATMSPGVRVAL
GCF_009618275.1_ASM961827v1_protein.faa
>Roseimaritima_sp._JC651
MGKPSKRQRAAMAKQPSEPQPLSDAVQSLQSFDKPKFDQTVEVHMRLGVDPNQADQIIRGSVVLPHGIGKQQRVVVFARGDAATAAEEAGADEVGQEDLAKKIKDGWTDFDVCIAAPDMMGLVGPLGRVLGPRGLMPSPRAGTVTPDVGKVVSEYKAGKVEFRNDKGGNVHAMVGKMSFDPKQLEENIQAFVDFVLGLKPQTVKGAYVRGVAICGTMSPSVRVAI
GCF_007741495.1_ASM774149v1_protein.faa
>Roseimaritima_multifibrata
MGKPSKRQRAALAKQPSDPQPLNEAVGTLKKYDKTKFDQTVEVHMRLGVDSNQADQIIRGSVVLPHGIGKTQRVVVFARGDAATAAQEAGADEVGQDDLAKKIKDGWTDFDVCIAAPDMMGLVGPLGRVLGPRGLMPSPRAGTVTPDVAKVVGEYKAGKVEFRNDKGGNVHAMVGKLSFEPTQLEENIQAFIDFVLGLKPQTVKGAYVRGIAICGTMSPSVRVVV
GCF_008065135.1_ASM806513v1_protein.faa
>Roseimaritima_ulvae
MGKPSKRQRASLAKQPSEPQPLNAAVESLKSYDKTKFDQTVEIHMRLGVDPNQADQII

GCF_007559335.1_ASM755933v1_protein.faa
>Rariglobus_hedericola
MPVKQSKRYVSAVKSADLTKGYSLAEATEVLSKFPKAKFDETVELSIRLGVDATQGDQMVRGTTPLPNGSGKKVRVIVFTDAPEVALKAGADEAGLAELMKKVSEGYLDFDVAIATTEAMKQVRTLARVLGPKGLMPNPKSGTVTDDIAAGIKAVKAGRVEFKMDKAANIGVGIGKRSFTSAQLVENAQAVLEAISKAKPASFKGTYIKSVGLSSSMSPGVRVASTEFTKL
GCF_001580045.1_ASM158004v1_protein.faa
>Cephaloticoccus_capnophilus
MDKKSKRYVSGTQVADLTKDYSLQEAVEILAKFPKTKFDETVELAFRLGVDPTAGEQMVRGTTPLPNGSGKTVRVAVFTDDAEAALAAGADHAGLQDLMAKVSEGWLDFDVAIATTEAMKQVRTLARVLGPKGLMPNPKSGTVTDDIAGGIKAVKAGRVEFKMDKSGNIALGVGKRSFTNEQILENAQTVIEAIGKARPATFKGSGYIKSIALAATMSPGIRLASSEFNKF
GCF_016595555.1_ASM1659555v1_protein.faa
>Cerasicoccus_arenae
MSKLSKRMAKANESLDLNKAYTLDEAVDTLKLFPTAKFDETIEVYAHLGVDPRQTTQMVRGTVALPHGSGKQVVVLVFTENPEEALAAGAEFAGLDDLIEKVLGGWMDFDVAIATPGAMKEVRKVARVLGPRGMMPNPKAGTVGEDIAKIVNEVKAGRVEFKMDKTANVAVVVGKRSFSKEQILENAQTAIDSLAKSRPDTFKGGAFLKSLTISSTMSPGVKIDVKALQLN
GCF_014230145.1_ASM1423014v1_protein.faa
>Pelagicoccus_albus
MVKLSKKFRAASEAADLSKEYTIAEAVEVLAKLPKAKFDETIEVSLKLG

GCF_000009945.1_ASM994v1_protein.faa
>Chlamydia_felis_Fe/C-56
MTKHGKRIRSILKSYDFSKSYSLQEAIDILKQCPTVRFDQTVDVSIKLGIDPKKSDQQIRGSVSLPNGTGKTLRILVFAAGEKAKEALDAGADFVGSDDLVEKIKGGWVDFDVAVATPDMMREVGKLGKVLGPRNLMPTPKAGTVTTDVVKAIAELRKGKIEFKADRAGVCNAGVGKLSFDGQLLKENIEALCSALIKAKPPAAKGQYLVSFTVSSTMGPGISVDTRELMAS
GCF_000454725.1_ibidis.assembly_protein.faa
>Chlamydia_ibidis_10-1398/6
MTKHGKRIRGILRTYDFSKSYSLGEAIDILKQCPTVRFDQTVDVSIKLGIDPKKSDQQIRGSVFLPNGTGKTLKILVFAAGEKAKEAVESGADFVGANDLVEKIKGGWLEFDVAVATPDMMREVGKLGKVLGPRNLMPTPKAGTVTTDVAKAVSELRKGKIEFKSDRAGVCNAGVGKLSFDGVQIKENIEALCSALIKAKPPAAKGQYLVSFTVSSTMGPGISVDTRELMAS
GCF_000470765.1_ASM47076v1_protein.faa
>Chlamydia_pecorum_PV3056/3
MTKHGKRIQGILKSYDFSKSYSLAEAIDILKQCPSVRFDQTVDISVKLGIDPKKSDQQIRGAVFLPNGTGKTLKILVFAAGEKAKEAVAAGADFVGSDEIIEQIKNGWLDFDVAVATPDMMREVGKLGKILGPRNLMPTPKAGTVTVDVVKAISELRKGKIEFKADRAGVCNAGVGKLSFDSAQIKENIEAFCHALIKAKPPAAKGQYLVSFTVSSTMGPGISIDTRELMAS
GCF_000008725.1_ASM872v1_protein.faa
>Chlamydia_trachomatis_D/UW-3/CX
MTKHGKRIRGIQETYDLAKSYSLGEAIDIL

GCF_007743815.1_ASM774381v1_protein.faa
>Alienimonas_californiensis
MGIRNYKPTSPGRRDASVADFAEITDRKKKPEKSLLVRITRKGGRNNQGVITARHRGGGARKMYRLVDFRREKDNVAAEVVAIEYDPNRSCRIALVQYADGEKRYILAPAKLEAGMTVMSAESGLDPDVGNCMPLSAIPLGTTVHNVEMRPGRGGQICRAAGTEAVLNARESSWAQLTLPSGEVRRVPAACRATIGAVGFSEHSAISLGKAGRKRHMGWRPRVRGTAMNPVAHPMGGGEGRNAGGRHPCSPTGVLAKGGNTRKRRKNSSKSIVRRRKSRRYGVQKI
GCF_007745435.1_ASM774543v1_protein.faa
>Calycomorphotria_hydatis
MGIRVYKPTTAGRRSASVADFAEITDRKKRPEKALTKRLRKTGGRNNQGKITVRHRGGGHKRLYRIIDFRRVKDDVQATVVGIEYDPNRSARIALLEYADGEKTYVIAPAGLKAGDKVMNGPNADPELGNCMPLSHIPLGTVIHNIEMRPGKGAQICRSAGTEAVLNAREENWAQVTLPSGEVRRIPSACRATIGAVGFSEHSSIRLGKAGRKRWLGRRPHVRGTAMNPVAHPMGGGEGRNSGGRHPCSPTGKLAKGGQTRKRRKASSKAIIRRRRSRRYGQLIT
GCF_007745175.1_ASM774517v1_protein.faa
>Caulifigura_coniformis
MGIRQYKPTSAGRRNASVSDFAEITDPKKKPEKSLLVRLKKHSGRNNQGKITARHRGGGAKKQYRIIDFKRTKDDIQATVGFIEYDPNRTCRIALLHYSDGEKRYILAPNGLKAGDKVMSGPKAEPKVGNTLPMSHIPPGTQVHNVELQPGRGGQICRSAGTAATMSARDGSWAQLTLPSGEIRRVPAACRATIGVVGNSEHSSIRIGKAGRNRHRGIRPHVRGSAMNP

...

GCF_008087665.1_ASM808766v1_protein.faa
>Mariniblastus_fucicola
MTITKEAKAEVIGEFQKNDKDNGSPDVQIAILTKRINSLTEHMRANPKDYSTRRGLLGMVSRRRRLLDYVRDHDPDRYLDLLERLGIRK
GCF_007860075.1_ASM786007v1_protein.faa
>Symmachiella_macrocystis
MSITKDRKSELIKEYARSDGDTGSPEVQIAVLTERINELTSHLRTHPKDHASRRGLLMLVSRRRGLLDYLKGKNQQKYVDIIGKLSIRK
GCF_013036045.1_ASM1303604v1_protein.faa
>Alienimonas_chondri
MSITKDRKQEVLSDFQRTDTDTGSSEVQIAVLTERINNLTEHLREHKQDHTSRRGLLALVSTRRRLLRYMQRTKPATYRELLTRLNLRK
GCF_007747795.1_ASM774779v1_protein.faa
>Maioricimonas_rarisocia
MSITKERKTELIGEFRRTDGDTGSPDVQIAVLTERINELTGHLKTHRKDHSSRRGLLMLVSRRRRLLDYVRRKDPGRYVDLITRLNIRK
GCF_900113665.1_IMG-taxon_2675903691_annotated_assembly_protein.faa
>Planctomicrobium_piriforme
MSVTKERKTRLISDFRQSTEDTGSPAVQIAVLTERINSLTDHLRSHMKDHAGRRGLLMMVSRRRRLLDYVKSEDPSLYVDLLKRLGIRK
GCF_008312935.1_ASM831293v1_protein.faa
>Roseiconus_lacunae
MTISKERKAEVIKEHGSQEGDSGSPEVQIAILTERINGLTEHMRTHHKDYASRRGLLGLVSKRRRLLDYVRGEDPQRYLDIIGKLGIRK
GCF_007475525.1_ASM747552v1_protein.faa
>Methyl

>Victivallis_sp._Marseille-Q1083
MDKAAKTAIVTKYARQEGDTGSPEVQVAILTQRISELTEHLRNNKKDHSTRRGLLAMVSRRKKLLAYLARTDHDRYLSLTDELAIRRK
GCA_012799725.1_ASM1279972v1_protein.faa
>Oligosphaeraceae_bacterium
MKKEQKEATIKEFQRSPNDVGSVEVQVALATGRIVELTEHMKIHKKDFSTRRGLVRQVNHRRNLLNYLQKRDYERYITLIRKLGLRR
GCA_018648945.1_ASM1864894v1_protein.faa
GCA_001803205.1_ASM180320v1_protein.faa
>Lentisphaerae_bacterium_GWF2_38_69
MKSDAKKQTVDKFALSKNDTGSAPVQVASLSTEIKSLTEHLKEHKKDHSSRRGLIAKVNKRRKLLNYFKREKPEECEKLIQDLKIRHQ
GCA_003635265.1_ASM363526v1_protein.faa
>Candidatus_Poribacteria_bacterium
MALAAAEKKELVNAYKTHDSDTGSPEVQVAIIQTRIRELTEHFRTHRKDHHSRQGLFRLVSKQRRLLRYLHRQDVQRYRNLITQLGIRDTISGRRN
GCA_001803235.1_ASM180323v1_protein.faa
>Lentisphaerae_bacterium_RIFOXYA12_64_32
MDKQAKAATIKEFARSEKDVGSAEVQVAVLTKRILHLTEHLKINRKDHSSRRGLTAMVNRRRKHLSYLDRTSHTRYLELIQRLGLRH
30S ribosomal protein S17 [ ### BREAK ###
GCA_016699285.1_ASM1669928v1_protein.faa
>Candidatus_Brocadia_sp.
MRSGNVRGRRKRVIGMVIGDKMQKTIVVEVACLIKHVKYGKYIKRSTIYKTHDENKQARVGDKVEI

>Coraliomargarita_sp._CAG:312
MAEESRKSRRSLVGVVTSLSGEKSIKVTYFYKVPHPLYGKEIRRKTVIHVHDENKECALGDKVEVMETRPISKLKRFRVVKVIEKAPQVASAE
GCF_003185655.1_ASM318565v1_protein.faa
>Coraliomargarita_sinensis
MEAVATRNSRKTLVGTVSSRSGDKTIKVTYSYKVPHPLYKKEIRRKTVVHAHDEKNECGLGDKVEIMETRPLSKLKRWRVTRVLEVAPKLGDEE
GCF_000171235.2_ASM17123v2_protein.faa
GCF_007559335.1_ASM755933v1_protein.faa
>Rariglobus_hedericola
MSTTTRNSRKDLTGFVTSRSGDKSIKVTVPYKIPHPRYQKTINRKTVVHVHDEKNETKVGDKVEIMETRPLSRLKRWRIVKVIEAAVAADGAAITEKDVAELVPTKKTKA
GCF_001580045.1_ASM158004v1_protein.faa
>Cephaloticoccus_capnophilus
MSQHIRHNSRKTLIGFVTSRSGDKSIKVTIPYKAPHPLYHKVVNRQTVVHAHDEKNETHPGDKVEIMETRPLSRLKRWRIVSILERAASIEDVAASVAESATAEEAAKA
GCF_016595555.1_ASM1659555v1_protein.faa
>Cerasicoccus_arenae
MSESQTQARSSHRKDLVGVVTSRTGDKSIKVTFFYKIPHPLYKKEIKRKTVVHVHDESNECAVGDKVEIMETRPLSKLKRWRVVKILDKAPIVG
GCF_014230145.1_ASM1423014v1_protein.faa
>Pelagicoccus_albus
MENSRNKRKDLTGVVTSIMGDKSIKVTYSYKKAHPRYRKVINRKTVVHVHDEENTARLGDKVEIMETRPLSRLKRWRLVSVVERAPMLGKKTLAEVEAEQEAHA

>Gemmata_obscuriglobus_UQM_2246
MSRSLKKGPHVDLRLLAKVEKQGQGKEPIKTWSRDCTIVPEFIGATFLVHNGKTFFKVYCTEDMVGHKLGEFSPTRTFKGHSGGKKAAAK
GCF_000186345.1_ASM18634v1_protein.faa
>Isosphaera_pallida_ATCC_43644
MGRSLKKGPYVDLRLLSKVEKAEAANQRDPIKTWSRACTIIPEFVGKNFAIHNGRTFIKLYVTEDMVGHKLGEFAPTRTFKSHGGKAKGKK
GCF_000196115.1_ASM19611v1_protein.faa
>Rhodopirellula_baltica_SH_1
MSRSSKKGPFVDPKVFFKVQKAAETGSKEPIKTWARSCTIVPEFVNVTFMVHNGRQHIKVLVTEDMVGHKLGEFAPTRTFKGHGGKGKR
GCF_000165715.2_ASM16571v3_protein.faa
>Rubinisphaera_brasiliensis_DSM_5305
MGRSLKKGPYVDHNLLGKIAKLDEAGRKEPIKTWARSSTIAPEFIGHTFLVHNGRAHVNVYVTEEMVGHKLGEFAQTRTFRGHGAKGKR
GCF_000092105.1_ASM9210v1_protein.faa
>Planctopirus_limnophila_DSM_3776
MSRSSKKGPYVDEKLLKKIEKLVVAGKKEPIKTWARRSTISPEFVGHTFLVHNGKQHLSVYVTEEMVGHKLGEFSLTRIFRGHGGKGNK
GCF_000255655.1_ASM25565v1_protein.faa
>Schlesneria_paludicola_DSM_18645
MSRSLKKGPYIDAGLISKIEKLDAVRKKEPIKTWARRSTISPEFVGHTFLVHNGKQHISVYVTEEMVGHKLGEFSLTRLFRGHGNKGNK
GCF_007750395.1_ASM775039v1_protein.faa
>Polystyrenella_longa
MGRS

>Pelagicoccus_albus
MARSIKKGFFVDPSLLKKVDAAQAANDRKPIQTWSRRSTITPEFVGLNINVHNGKAFIPVHVTENMVGHKLGEFSQTRTFKAHGGGKK
GCF_014803405.1_ASM1480340v1_protein.faa
>Pelagicoccus_enzymogenes
MARSVKKGFFVDPSIEKKVKEAQASNDRKPIQTWSRRSTITPDFVGLNINVHNGKAFIPVHVTENMVGHKLGEFSQTRTFKAHGGGKK
GCF_016595505.1_ASM1659550v1_protein.faa
>Pelagicoccus_mobilis
MARSVKKGFFVDPSLDKKVAAAQAANDRKPIQTWSRRSTITPDFVGLNINVHNGKMFIPVHVTENMVGHKLGEFSQTRTFKAHGGGKK
GCF_014230085.1_ASM1423008v1_protein.faa
>Ruficoccus_amylovorans
MARSIKKGFFVDPKLMDKVEKAQAEGTHKPFQTWSRRSTITPDFVGLNINVHNGKGFIPVHVTENMVGHKLGEFAPTRSFKGHQPHTKK
GCF_001318295.1_ASM131829v1_protein.faa
>Candidatus_Xiphinematobacter_sp._Idaho_Grape
MGRSLKKGPFVEWKLLEKIDDLNSSGTRKSIKTWSRKSTITPDFVGHTFNVHNGKVFVPVFVTENMVGHKLGEFSPTRIFRKHGSHTAKATK
GCF_004341915.1_ASM434191v1_protein.faa
>Chthoniobacter_flavus
MGRSLKKGPFVDGHLFEKIDKMNDSGAKKPIKTWSRRSMITPEFVGHTFLVHNGKIFNSVFVTENMVGHKLGEFSPTRIFKKHGSHTAKVAK
GCA_002279575.1_ASM227957v1_protein.faa
GCF_001613545.1_ASM161354v1_protein.faa
>Terrimicrobi

In [33]:
####

In [34]:
protein_name='30S ribosomal protein S2 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S2 [
GCA_016699285.1_ASM1669928v1_protein.faa
GCA_013359215.1_ASM1335921v1_protein.faa
>NUM34872.1 30S ribosomal protein S2 [Candidatus Brocadiae bacterium]
 MAIISVKSLLEAGVHFGHRVSRWNPKMAPYIFGRRNSIHIINLRETVKGLVRAYYFIQKLTSEGGEVLFVGTKRQAKAVIRRESERSKMHYVAERWLGGSLTNYTTIRKRLARLEELEGMENDGSMSSLSKKLGSTLSREKKKIHRNLEGIRNMERMPDALILVDPQKEHIALKEAVKMGIPTIALMDTDSDPDLIDIPIPGNDDAMRSIEVVCSKLADAIVSGKAIWEEKRRLEERKREESTRQSSTVRQQAETSDGSKYATGSTNPAAPKVEDKKDDAKKAEDRKRNRRGKPRPGSGKPFEAKADGATAPKDGGEKKEFSGGPKRPPFKGKRPDNPRRDNAAQTKEQPKEVAPKAETKPVEVTSNMEKTQAEKEN
GCF_000284115.1_ASM28411v1_protein.faa
>WP_014437511.1 30S ribosomal protein S2 [Phycisphaera mikurensis]
 MADSTPASPASSSPASLVKSLIDAGIHFGHRANSWNPKMTPYIFGKRNRIHIIDVKETIKGLLLARKFITKEVAAGKDVLFVGTKRQARSIVESHVKECGMHYVTERWLGGTLTNFSTIRQRLKRLEELEQLEESGEIANYSKKMEATLNREKSKIKRNLEGIRNMGKLPGLMVIIDVNAEQNAVKEARKLGIPIVALLDTDSNPDDADIAIPGNDDAMRAIEIVVDQLARAANEGRGKRPQSAPSAEGMVDGDGNQAPGDPRGPKGPGGRRSNRAQFSAKDTPGETAEEAQGGTAAEPIDDGKVDGATADKPAHQPAATV
GCA_016932895

GCF_007859755.1_ASM785975v1_protein.faa
>WP_146436889.1 30S ribosomal protein S2 [Blastopirellula retiformator]
 MSTDGSANSVIKELVESGIHYGHRTSRWNPKMAPYIYAKKNQIHIIDVRETIRGLLRAKKYLTQVSEGGSLVLFVGTKRQAGAAIEREAERCGMPFINERWLGGTLTNFRTIRSRLTRLEELESILEGDELHKYSKKMQSSLAREHRKMYRNLNGLRTMNRLPETLVIIDPKKERNAIREAQSLGITTLALTDTDCDPDEIDLPVPGNDDGIRSIELFVKHLADAVIDGKHNIAMKGKDAAPAS
GCF_009618275.1_ASM961827v1_protein.faa
>WP_153557583.1 30S ribosomal protein S2 [Roseimaritima sp. JC651]
 MIEAGVHFGHRTSLWNPKMAPYIFGRKNQIHILDIRETLRGLLRAKKYLQQVAENNGLILFVGTKRQAGEAVEEHCLRCGMPFVAERWLGGTLTNFRTIRSRLSRLEELEKIRSGDELDNYSKKMQSALNREYRKMYRNLNGLRNMNRLPECLVIIDPGKERNAVREAKRLGIPTVALIDTDSNPDLIDLPIPGNDDGIRSIEIIMKQLADAVIAGKGTPAEAEKAGAPAGASEQAAEATS
GCF_007741495.1_ASM774149v1_protein.faa
>WP_145355463.1 30S ribosomal protein S2 [Planctomycetes bacterium FF011L]
 MIESGVHFGHRTSLWNPKMAPYIFGRKNQIHILDIRETLRGLLRAKKYLQQVAESGGLILFVGTKRQAGEAVEEHCLRCGMPFVAERWLGGTLTNFRTIRSRLGRLEELEKIRSGEALDKYSKKMQSALTREYRKMYRNLNGLRSMNRLPECMVIIDPGKERNAVREAKRLGIPTV

GCF_007475525.1_ASM747552v1_protein.faa
GCF_000297415.1_ASM29741v1_protein.faa
>WP_009059159.1 30S ribosomal protein S2 [Methylacidiphilum fumariolicum]
 MVELEDLIKSGTYFGHRKDRWNPKMAPFLLGLKGGIHLINPNKTMECLKRAADFLKKTSIEGGKVLFVGCKRQAQGLIEEIAKKSGSFYVNYRWLGGTLTNMVTIRKSIARMKWIDQLEASGTMERLPKKEVAKLRRENAKLHRSLDGIKDMERLPAAIVIVDVVKEEIAVSEARKLDIPIVAIVDTNGNPENIDYPIPANDDSIRSIRTILEEIGNAIIEGKKEGGFSLPVENAGNITSEAAILSTN
GCF_000019965.1_ASM1996v1_protein.faa
>WP_012377286.1 30S ribosomal protein S2 [Opitutus terrae]
 MNVTPKDLLDAGVHFGHQTKRWNPRSKPFVFDHRQGISIIDLGKTHAALEKACTFLEDTVGNGGNVLFVGTKRQAKDIIREAATSTNMPFCVDRWLGGTLTNYETVKRSIAKYKKYQQMETSGDMNKLASKEVAAIKREMVRMQKNFSGIVDMPGLPTAMFVVDVNHEKIAVAEAARSGIPCVGICDTNSDPSTLSHPIPGNDDAVKSIRIIVEAIVAAVQNGLSQRDARRAARGAADLKAAAAAAAGITGETAATPEVDLSKVELPADVAAVVEGEGESEAEPVVAKKKPVRAKRPAVKAE
GCF_003096195.1_ASM309619v1_protein.faa
>WP_108823889.1 30S ribosomal protein S2 [Ereboglobus luteus]
 MNVTPKDLLDAGVHFGHQTKRWNPRSKPYVYDHRQGITIIDLGKTHELLQKAYTTVEDTIANGGTILFVGTKRQAQEIIKEAAASTNMPYCVD

>WP_079978015.1 30S ribosomal protein S2 [Criblamydia sequanensis]
 MPKSIFKEGQGLDEQTQQDQISVSIKDLLEAGSHFGHQTHRWNPKMKRFIFEERNGLYIIDLAKTLQLLRSAVGFIKEIVGKHRSVLFVGTKKQAKEVLKDLAEKCGEFYVTERWLGGMLTNLSTIRQSIKKLERIEKKISTQSDTLTKKELSLLTKDQIKLDKNLSGIRAMRKPPGLVVVVDPGKEHIAVAEANKLGIPVMALVDTNCNPDPIQYVIPCNDDALKSIKLILEALAQGIIDKKNEMQVGFGKEEDSEESESMAASKKYEGALAGEGDKD
GCF_900000175.1_ASM90000017v1_protein.faa
>WP_098038698.1 30S ribosomal protein S2 [Estrella lausannensis]
 MATQNQEQISVSIKDLLEAGAHFGHQTHRWNPKMKRYIFEERNGLYIIDLAKTLHLLRNATNIVKEVVEKHRSILFVGTKKQAKDVVKELAEKCGEFYVSERWLGGMLTNLNTIRQSIKKLEKIEKKISTQADTITKKELSELTKQQIKLEKNLSGVRGMRKPPGLVVVVDPSKEHIAVAEANKLGIPVMALVDTNCNPDPIQHVIPCNDDALKSIKLILDALAQAVVAKKGEMQIGIGKQTEGEEDSESAASKRMEDALASQEGE
GCF_015356815.2_ASM1535681v2_protein.faa
>WP_194845151.1 30S ribosomal protein S2 [Candidatus Rhabdochlamydia porcellionis]
 MPSSHKGQSLEQQEQYKEVTIKTLLESGAHFGHQRHRWNPKMKRFIFEERNGIYIIDLSKTLQQIRNATQVVMNTVSKRKSILFVGTKKQAKNLVRECAKQCEEFYVCERWLGGTLTNLTTIRQSIKKLERIEKKIALNADDLTKKELSLLT

GCA_003635265.1_ASM363526v1_protein.faa
>RKU07739.1 30S ribosomal protein S2 [Candidatus Poribacteria bacterium]
 MRTVTMKELLEAGVHFGHQSRRWNPKMAPYIFAERNGIYIIDLQKTLRQLQTAYEFVQEVAARGGAVLFVGTKKQTQETVAEEAARCQMFHVNHRWLGGMLTNYETIRRSINRLMKIEEDEANGVFEQLPKKEVMSLRREKTKLDNNLAGIKTMGKLPEAVIITDTRKEHIAVKEANKLGVPVVAIVDTNCDPDPIDYPIPANDDAIRAIKLICSVLATAVIEGRASALEGADDAEPSLEEASEPEASESEAAADEGGVENQESEDSQEDSQEKSQEDSQEEDQEDNS
GCA_001803235.1_ASM180323v1_protein.faa
>OGV68288.1 30S ribosomal protein S2 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MATIAITDLLEAGVHFGHQTKRWNPKMKPYIYGARNGINIFDLTITMRQLAEACSFLTKTVAGGGSVLFVGSKRQAQEVVREAAERTGMFFMCDRWLGGVLTNHQVITTRIQYMKKLREITAEGSTEKLTKKEMAKLRRILEKLERTLGGIAEMGRLPSIMVVVDTGREAIAVREANRLDIPVVGIVDSNSDPELIEYVVPGNDDALRAIKVVIDTLAAAIEEGRQSRSKDRNEPVPAARKTNAKADAAPVEAVAAATVEAPAPVELSADEVEKAAAVQ
102


In [35]:
protein_name='30S ribosomal protein S4 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S4 [
GCA_016699285.1_ASM1669928v1_protein.faa
>QQR67146.1 30S ribosomal protein S4 [Candidatus Brocadia sp.]
 MARYVGPQCRLCRREGEKLFLKGIRCDTVKCAITKRKYPPGQFTWGKGKLSKYGIQFREKQKVKRYYGILEKQFKNYFKKAERQKGNTGENLLLMLERRLDNVVHLLSFAPSRKSARQFVLHGHIMVNNKKVDIASYLVKTGDIIKPRSGEITQNIVKTNLEMVKGRNVPAWMQLRVDVLEGLVTQLPTREDISVPAQEQLIVELCSK
GCA_013359215.1_ASM1335921v1_protein.faa
>NUM34075.1 30S ribosomal protein S4 [Candidatus Brocadiae bacterium]
 MGRYIGSVCKRCRREGVKLFLKGERCNSGKCSLAKRENPPGMHNWRKGKLSDYGVRFREKQKVKRYYGVYDRQFKNYFALATKQRGNSGENLLVLLERRLDNVVCRGSLAASRPQARQFINHGHILVNGKKVDIPSFLVKEGDVITHVNHEISKKMIQQVYESLAAREVPSWIQKTPDKAEIKITHLPVRAEIGLEINDQHIVEFCAR
GCF_000284115.1_ASM28411v1_protein.faa
>WP_014436112.1 30S ribosomal protein S4 [Phycisphaera mikurensis]
 MANYTGPKVKLSRRVGVPIADLPKHTTKRQLNPPGMHGFRARRLRDYGVRLNEKQKLRYHYNVLEKKFRLYLDEAARSKGNTGVLLLQLLETRLDNAVRRMGWTRSIWAARQMVGHGHVLVNGKRVDIPSYRLLVGDTVEVRTDKAKKLARENMESLAGLEVPEWLNANPAELSCKIASLPSVDEVPFDVNMNLIIEFYR
GCA_016932895.1_ASM1693289v1_p

GCF_009618275.1_ASM961827v1_protein.faa
>WP_202617313.1 30S ribosomal protein S4 [Roseimaritima sp. JC651]
 MARYTGPKARINRRLGALIYETAGASKALDRRPNPPGMHTRGRRPSNYGIALMEKQKIKHYYGLGERQLRRYFDAVSRRSGNTGELLLLMCESRLDNVVRRCGFTKTRPQARQGIVHGHFRVNGVKVTKPNYLLRPGDVIEVRNRENLKDVYRGVIANATPDALDWVAFDSENLKATVQGLPGPSDISLPVDANAVVEFLSR
GCF_007741495.1_ASM774149v1_protein.faa
>WP_145352780.1 30S ribosomal protein S4 [Planctomycetes bacterium FF011L]
 MARYTGPKARINRRLGALIYETSGASKALDRRPAPPGMHTRGRRPSNYGIALMEKQKIKHYYGLGERQLRRYFDAVSRRSGNTGELLLLMCESRLDNVVRRCGFAKTRPQARQGVVHGHFFVNGTKVAKPNYLLRPGDIIEVRNREKLKDVYRGVIANSSPDALDWVAFDSENLKATVQGLPGPSDISLPVDANAVVEFLSR
GCF_008065135.1_ASM806513v1_protein.faa
>WP_068137083.1 30S ribosomal protein S4 [Roseimaritima ulvae]
 MARYTGPKARINRRLGALIYETAGASKALDRRPNPPGMHTRGRRPSNYGIALMEKQKIKHYYGLGERQLRRYFDAVGRKSGNTGELLLLMCESRLDNVVRRCGFTKTRPQARQGIVHGHFRVNGVKVTKPNYLLRPGDVIEVRNRENLKDVYRGVIANATPDALDWVAFDSENLKATIQGLPGPSDISLPVDANAVVEFLSR
GCF_008035925.1_ASM803592v1_protein.faa
>WP_147870255.1 MULTISP

>WP_191619612.1 30S ribosomal protein S4 [Pelagicoccus enzymogenes]
 MARYTGPTTKISRRFGQPIFGSSKSLEKRNYPPGQHGPRLRRKQSEYSIGLGEKQKLRFMYGLMEKQFRRTFDKAKATRGVTGTIFLQLLETRLDSIVYDLGFTKTRKAARQFVNHGHICVNGKKVDIPSYQVKVGDEIEVRNATSSRQLATRNMEANRMRNVPDWMTRQDEAFRGVVNREPTREELEPSINEQLIVEFYSRF
GCF_016595505.1_ASM1659550v1_protein.faa
>WP_200356080.1 30S ribosomal protein S4 [Pelagicoccus mobilis]
 MARYTGPTTKISRRFGQPIFGSNKSLEKRNYPPGQHGPRLRRKQSEYSIGLGEKQKLRFMYGLMEKQFRRTFDKAKNTRGVTGTIFLQLLETRLDSIVYNLGFTKTRKAARQFVNHGHICVNGQKVDIPSFQVKVGDEIEVRNSTSSRQLATRNMEANRLRNVPDWMTRTDEAFRGVVNREPTREELEPSINEQLIVEFYSRF
GCF_014230085.1_ASM1423008v1_protein.faa
>WP_185674309.1 30S ribosomal protein S4 [Ruficoccus amylovorans]
 MARYTGPTTRINRRFGQAIFAPSKAFERKPHPPGIHGPRLRRKTSDFSTGLNEKQKLRYMYGLTEKQFRRTFDKAKAQRGVTGEIFLQLLETRLDNVIYQLGLARSRQAARQFVCHGHIQVNGHKVDIASYEVKPGDIIEVRDRTSSRQLATRNLEDSQYRTIPAWLALDHDALRATANRLPVAEEMEQGINLQVIVEFYSR
GCF_001318295.1_ASM131829v1_protein.faa
>WP_062100529.1 30S ribosomal protein S4 [Candidatus Xiphinematoba

>RKU11398.1 30S ribosomal protein S4 [Candidatus Poribacteria bacterium]
 MSKYLGSVCKLCRREGEKLFLKGQRCDSPKCSFERRSYPPGQHGQKLPKVKDYGRQLRTKQKSKRIYSILEKPFKNYYTKAAHQKGVVRENLLTLLERRLDNVIYRLGFANSRLQARQLVTHNHFVVNGKRVNVPSYLIQPGDVIQTREKSRNVAVIQEALERSQHRGTPDWLELDIDKQEGVVRMFPNVEQLATDLQTQLIVELYSK
GCA_001803235.1_ASM180323v1_protein.faa
>OGV64092.1 30S ribosomal protein S4 [Lentisphaerae bacterium RIFOXYA12_64_32]
 MKRGPKHKLCRRVGSCIWGSPKCPSNKRPYAPGPHGNQRRRTKLSTYGILMMEKQKLRALYDLNEQELVFAYRKAKAGTGSTGEKLLRNLEMRLSSIVYRSGLAPTVFAAKQAISHRHILIDGKVVNRCSCRVKPGQVVSVNAQRSPAIANIAQKTDIVAPPYLELEKENCKVTVAREPLTEEIPANIEIMRVVEFYAR
105


In [36]:
protein_name='30S ribosomal protein S5 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

30S ribosomal protein S5 [
GCA_016699285.1_ASM1669928v1_protein.faa
>QQR67138.1 30S ribosomal protein S5 [Candidatus Brocadia sp.]
 MARLEEPKNFEETVVRINRCTTVTKGGKSMSFSALVVVGDKKGSVGIGFGKAREVPNAVNKAVKEAKKQMVKIPLRGDTIPHVIWGRFKSANVFLKPASPGTGIKAGASARAVLESVGIKNILSKCYGKRNPLNVVKATLMGLKALRTKEEVGVLRGVKIE
GCA_013359215.1_ASM1335921v1_protein.faa
>NUM34067.1 30S ribosomal protein S5 [Candidatus Brocadiae bacterium]
 MKPPREEEIQQDYVEYVVKINRCATVVKGGRRFSFSALVVVGNKNGKVGIGFGKANEVPPAVQKAVKDAKKNSVKLDLCGDTIAHEVIGKFGATEVFMKPAAPGTGVIAGAAVRAIAEAAGIKNILSKAYGSTNPVNLTKAALKGLQSIRSKEQLRRLRGVEIK
GCF_000284115.1_ASM28411v1_protein.faa
>WP_014438268.1 30S ribosomal protein S5 [Phycisphaera mikurensis]
 MAETLDGNQLESTTVAINRTSTTVKGGRRLSFSALVVVGNRDGKVGIGYGKGRGVPVGIEKAQKVARKEMVAIKRLGATVPHEVIGRSCASTVKLIPAAPGTGVIAGGTVRAVLEMAGVRDCLTKAYGSTNKINLCRAVMDALDQLRTREEIAKLRGVEIEKSTVDEALEANKRYMTEEGDAGPKKKAKGPTNQKDEDAKKTRGRGGRGGPGGGRGGNAGGPAAPKPDADKDAQTADGEGNVATGIAGSVAGAAGAVASVAGGAVGAVLGAVTGGSDTAGDKPETTDGDDVGAAAAPATPDAADVPRDAETEEGKA
GCA_0

GCF_007741495.1_ASM774149v1_protein.faa
>WP_145351112.1 30S ribosomal protein S5 [Planctomycetes bacterium FF011L]
 MVSQSNQPGELLDRVVKIKRCAAVVKGGRRFSFAAMVVVGDGKGRVGWGYGKANEVPPSVQKGQKQASRSLINVPVVEGSIPHQVWGRFGAAKVLLIPAGAGTGIIAGQAVRSVCEACGIHDILTKSYGTNNPVTLVKATIDALSKLRTRDEVAALRGLDPDELRGETSSAPVKAAAPVQSA
GCF_008065135.1_ASM806513v1_protein.faa
>WP_068136940.1 30S ribosomal protein S5 [Roseimaritima ulvae]
 MNVSQSTQPGELLDRVVKIKRCAAVVKGGRRFSFAAMVVVGDGKGRVGWGYGKANEVPPSVNKGQKQASRSMIDVPVVDGTIPHQVWGHFGAAKVLLIPAGAGTGIIAGQAVRSVCEAVGLHDILTKSYGTNNPVTLVKATIDALSKLRTREQVAALRGLDSAELMGATAPAPVKTAAAAE
GCF_008035925.1_ASM803592v1_protein.faa
>WP_199773811.1 30S ribosomal protein S5 [Stieleria maiorica]
 MLDRVVKIKRCAAVVKGGRRFSFAAMVVVGDGQGKVGWGYGKANEVPPSVQKATKQATRDMVNVPLVEGSIPHQVWGSFGAAKVTLIPAGAGTGIIAGQAVRAVCEACGIHDILTKSYGTNNPVTLVKATVEALKQLRTREETAALRGLSVEELSA
GCF_007860045.1_ASM786004v1_protein.faa
>WP_231741757.1 30S ribosomal protein S5 [Stieleria varia]
 MADGLLDRVVKIKRCAAVVKGGRRFSFAAMVVVGDGRGRVGWGYGKANEVPPSVQ

GCF_016595505.1_ASM1659550v1_protein.faa
GCF_014230085.1_ASM1423008v1_protein.faa
>WP_185674315.1 30S ribosomal protein S5 [Ruficoccus amylovorans]
 MSNQPSSNNPHERNRGGQGRGGQGRGGPGRGRGPRRERASEPVEEQQYVEKVVHINRCAKVVKGGRRFSFAALVVSGDGKGKVGIGYGKAKEVPEAIRKGTERARANQEDICLKGRTIPHEVRGEHDGGLVILRPASEGTGVIAGGGIRAVLEAVGIQDVLSKSLGSNNPGAMVNATLEGLRQLRSAEQIARIKQAGTKKEEPAATLA
GCF_001318295.1_ASM131829v1_protein.faa
>WP_082383022.1 30S ribosomal protein S5 [Candidatus Xiphinematobacter sp. Idaho Grape]
 MLTPKNSKERRNTSRPPRGANRQAADSKNTLTDKVVFINRCAKVVKGGRRFSFSALIVSGDRQGRVGVGFGKANEVSEAIRKATVAAHKCLLNVAITANTLPHEVVGEFGGGRVLLRPASPGTGVIAGGGVRAMMEVVGIRDVLAKSLGSSNPTNVVKATLNALTTLRKREQVDQVRGKTTPFQGFLAAG
GCF_004341915.1_ASM434191v1_protein.faa
>WP_006977830.1 30S ribosomal protein S5 [Chthoniobacter flavus]
 MAEQRQDRGNRGRRDRQEKPKDDSGLTEKVVFINRCAKVVKGGRRFSFSALIVTGDKLGKVGVGFGKANEVSEAIRKASDSAKKAMVKVAIRENTIPHDVIGEHGGGRVLLRPASPGTGVIAGGGVRAVVEAAGIRDVLAKSLGSSNHANVVKATIGALQQLRLREQIFETRGLTVKKAQA
GCA_002279575.1_ASM227957v1_protei

GCA_001803235.1_ASM180323v1_protein.faa
>OGV70023.1 30S ribosomal protein S5 [Lentisphaerae bacterium RIFOXYA12_64_32]
 METQRTGERTGGRTAERTGECEERVVCINRSSKVVKGGRNFSFSALIVAGDRGGHVGVGFGKSNEVADAIRKGSELARKSLCKVPMRGQTIPHAVTAKFRSTKVLIRPASPGTGIIAGGGVRSVLELCGVRDVLSKVLGSKNPVNVVKATMLAISRLRTKQEILAKRDITALTL
104


In [37]:
protein_name='50S ribosomal protein L10 ['
num_complete = 0
print(protein_name)
for index,row in taxa_df[:].iterrows():
    print(index) #Turn this on to get specific.
    index_complete = 0
    for header, seq in fasta_iter(index):
        index_complete = index_complete+1
        if header.find(protein_name) != -1:
            print(header,seq) #Turn this on to get specific.
            num_complete = num_complete+1
        else:
            continue
            print('ERROR!',index,protein_name)            
print(num_complete)

50S ribosomal protein L10 [
GCA_016699285.1_ASM1669928v1_protein.faa
>QQR67114.1 50S ribosomal protein L10 [Candidatus Brocadia sp.]
 MTNELKQSVVKELISNYRDKNNYLVIGYQGIKALEFDQLRKDLRKKKICIGIVKNSLAGIAFKEIGVRGVVDLLKGPTAIVTGGGDPVVMAKETIEWTKKIPFLSLRGGLVDGAMLSVHDINDLAKLPSMPVLRAQIITGIHAPIAGVAGAFHAVLRGLVTVFQAVKDKKEKSGV
GCA_013359215.1_ASM1335921v1_protein.faa
>NUM33276.1 50S ribosomal protein L10 [Candidatus Brocadiae bacterium]
 MSKLLRKLMAKELSKTLKNIDCFILFGYDRVNAEQAHQFRVKLRENKVSVKILRNKLSRIAIEQTYNKNLKNLLKGPVGIVYGGESPVDLAKIMTEWAKKNKTTPIKGGFLSGQVLSVKDIDELSKVPPREVVLSQLAGAFVGSIQQVATIFSAAIQDASNAFGSLAEKLEKQEKPS
GCF_000284115.1_ASM28411v1_protein.faa
>WP_014436227.1 50S ribosomal protein L10 [Phycisphaera mikurensis]
 MSKPVKNLITDQYRELFKDVGGAVVVDIRGIPANTTNAMRADLASKNIKVTIVKNSLAKRAWDGHALEPMGDYLDGACTVVYPTDENASAVVVARELVDWAKKEKKIQFRGAVLDGLKFGPEEITRLSEFPTKAEAQATVVTLLLSPAKNLVGATLAPARNIAGILKTVQEKLEAGETLAKAG
GCA_016932895.1_ASM1693289v1_protein.faa
GCF_015169395.1_ASM1516939v1_protein.faa
>WP_206292234.1 50S ribos

GCF_007741495.1_ASM774149v1_protein.faa
>WP_145350817.1 50S ribosomal protein L10 [Planctomycetes bacterium FF011L]
 MSKFVKNLVTRDIAGRLEGVADAVVVSTVGMDANTTVELRNELAEKDIHMLMVKNSLARRATDGTSLAPAFEGANGPVAIVWGSTDYVSLVKAVVALDKDGDKYDHFKAQCGAMDGEALDADALKAVSKWPSREEQISTLVGQILAPGANLSGALIGTAKGLASQVKQIMEKAEE
GCF_008065135.1_ASM806513v1_protein.faa
>WP_068136897.1 50S ribosomal protein L10 [Roseimaritima ulvae]
 MSKFVKNLVTRDISNRLEGVEDAVVVSTVGMDANTTVELRTELAEKDVQMMVVKNSLARRASEGTKLAPAFEGASGPIAVCWGSTDFVSLVKTIVALDKDSDKYELFKAQGGAMDGEALDAEGLKAVSKWPSREEQISLLVGQILAPGANLSAALLGPAKMLASQVKQIEEKAEDSAE
GCF_008035925.1_ASM803592v1_protein.faa
>WP_147865956.1 50S ribosomal protein L10 [Stieleria maiorica]
 MSKYVKQLVTRDIQRQLDGVQDAVLVSTVGMDANTTNELRGELAERNITMFVVKNSLARRATEGTSLAAAFEGASGPVAVCYGGEDFVSLVKEVVRLDKDDDKYEKFSTTGGVMDGERLDADGVKAVSKWPSREEQISMLVGQILGPGSTLSAAMLGPGKTLNSQLKKISEGDE
GCF_007860045.1_ASM786004v1_protein.faa
>WP_146523109.1 50S ribosomal protein L10 [Stieleria varia]
 MSKYVKQLVTRDIQRRLDGVKDAVLVTYPGMDANTTNELRTEL

>WP_191619648.1 50S ribosomal protein L10 [Pelagicoccus enzymogenes]
 MRPEKQFLVDEINTHLDKSDYLFLADYERSTVIDIAELRAELIKEEAEFHVVKNNILKVAARERGYPEIDEHLNGQNAIVIGGTNPSGVAKVLTKFFDKKDKMDVKVGILSSQRLEKDEVVALSKLPNIDAMRAQLLGLLSQPAQSLVFVLNGVPTAMLNVLQAKADKGE
GCF_016595505.1_ASM1659550v1_protein.faa
>WP_200354950.1 50S ribosomal protein L10 [Pelagicoccus mobilis]
 MRPEKKFLVDEVNTHLDKSDYLFLADYERSTVIDIAELRAELTKEEAEFHVVKNNILKVAARDRGYPEIDEHLTGQNAIVIGGNNPSGVAKVLTKFFDKKEKMDVKVGILASQRLEKDEVVALSKLPGLDALRAQLLGLLSKPAQSLVFVINAVPQNVLNVLQAKVDAEGGE
GCF_014230085.1_ASM1423008v1_protein.faa
>WP_185674342.1 50S ribosomal protein L10 [Ruficoccus amylovorans]
 MRPEKQFLVDEVNTHLDKSEYVFLTDFSRLNVEDTAQLRGLLAAKDAEFHVVKNNIFKVASRARELPVDESVLNGQTAIVVGGPNPSEVAKVLFKFFKDKEKVDIKTGILGDKALTRDDVEALSKLPSLEALRSQLLGLFTQPGTQMVRVLIAGPQDFVNVLQAKVRAEGGEA
GCF_001318295.1_ASM131829v1_protein.faa
>WP_062100385.1 50S ribosomal protein L10 [Candidatus Xiphinematobacter sp. Idaho Grape]
 MRPEKTSIAADIQNCLKQSSFLFMVGYAGMQVAHFDELRKRLKAVNAKLCVVKNTLLLRAMQGLGF

GCF_000454725.1_ibidis.assembly_protein.faa
>WP_020370397.1 50S ribosomal protein L10 [Chlamydia ibidis]
 MKEEKKLLLQEVEEKISASQGFILLRYLGFTAVHAREFRNSLSGVCAEFEVLKKRIFFKAAEAAGFEVNGSDMEGHLGVVFAYDDPVSAAKRVLDFNKQHSDSLVFLAGRIDNASLSGKEVEAVAKLPSVKELRQQIVSLLAAPMSQVVGVMHSALSGVILCIDQKAEKNKE
GCF_000470765.1_ASM47076v1_protein.faa
>WP_013713063.1 50S ribosomal protein L10 [Chlamydia pecorum]
 MKEEKKLLLQEVEEKISAAQGFIVLRYLGFTAAYARELRNSLFEVSAEFEVLKKRILFKALESQGVQYDCFNADGHLGVVFSYKDAVSAAKQVLDFNKQHGDSLVFLSGRIDNAFLSGEEVEAIAKLPSLKELRQQVVGILAAPMSQVVGVMGGALSGVLSCVEQKIEKTQE
GCF_000008725.1_ASM872v1_protein.faa
>NP_219822.1 50S ribosomal protein L10 [Chlamydia trachomatis D/UW-3/CX]
 MKEEKKLLLREVEEKITASQGFILLRYLGFTAAHSRSFRNNLSGVSAEFEVLKKKIFFKALETSGVEMDPEGGEGHLGVVFAYGDPVSAAKQVLDFNKQHNDSLVFLAGRIDNASLSGREVEAVAKLPSMKELRQQVVGLIAAPMSQVAGIMNSVLSGVISCVDQKAEKTQE
GCF_003096415.1_ASM309641v1_protein.faa
>WP_116885062.1 50S ribosomal protein L10 [Victivallis vadensis]
 MRNEKHFLVKQISDMIGNADYVYFISFAGLTVKDFSELRNQLAAQNAVCHVLKNTL

In [38]:
### Changes implemented for
# Coraliomargarita_sp._CAG:312
# Gemmata_obscuriglobus_UQM_2246 

In [39]:
taxa_df.loc[taxa_df.Name=='Coraliomargarita sp. CAG:312']

,Index,Name,URL,Filename,FullURL,Accession,TaxID
Filename,,,,,,,
GCF_000438015.1_MGS312_protein.faa,56,Coraliomargarita sp. CAG:312,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCF_000438015.1_MGS312_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,GCA_000438015.1 MGS312,1262865


In [40]:
checkGenome('GCF_000438015.1_MGS312_protein.faa')

50S ribosomal protein L1 [
>WP_022478266.1 50S ribosomal protein L1 [Coraliomargarita sp. CAG:312]
 MVKRSKRYKKALENLGDAPKSLADAVKALCAMPKAKFDETVEVSFKLNVDPKQSTQMVRGTVRLPHGSGKKVRVVVFTEHADQALAAGADKAGLEDLIKEVEGGWMDFDVAIATTSAMKEVRKVARVLGPRGLMPNPKSGTVTDDIVSAINEVKAGRVEYKMDKTANLTVVVGKRSFSEDKLVENAKTALSSVAAARPESIKGKFINSVTISSTMSPGVKIALSEVAE
1
50S ribosomal protein L2 [
>WP_022478866.1 50S ribosomal protein L2 [Coraliomargarita sp. CAG:312]
 MAIIKSRPLTPSQRFLELGRNEVDNKRPERSLTESKQRAKGRNCYGRITSRRRGGGHKKLYRIVDFKRDIVDVPATVVAIEYDPNRSAYLALLNYENGQKRYIIAPDGLKKGDTVLTTSKKQSEFPVGLCLPLKLIPPAMKIHSLEMQPMKGAQIARGAGAGAQLVAIEGEKATVKMPSGELRIFNADCRAVIGVVGNIEYRNQSLGKAGRVRWMGRRPRVRGVVMNPVDHPMGGGEGRTSGGGHPVSPWGQLSKGYPTRRKSKPSNSMILLRRNGKKAKK
1
50S ribosomal protein L3 [
>WP_022478863.1 50S ribosomal protein L3 [Coraliomargarita sp. CAG:312]
 MPGHMGQVRCTTQNHPIVKILEDKNILLIKGSLPGAKGDLVIVRPAKKVKIAK
1
50S ribosomal protein L4 [
>WP_022478864.1 50S ribosomal protein L4 [Coraliomargarita sp. CAG:312]
 MKLKVYTKDGSSFVEQDFENIPQFEGDKG

In [41]:
taxa_df.loc[taxa_df.Name=='Gemmata obscuriglobus UQM 2246']

,Index,Name,URL,Filename,FullURL,Accession,TaxID
Filename,,,,,,,
GCF_901538385.1_ASM90153838v1_protein.faa,6,Gemmata obscuriglobus UQM 2246,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...,GCF_901538385.1_ASM90153838v1_protein.faa,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...,GCA_901538385.1 ASM90153838v1,114


In [42]:
checkGenome('GCF_901538385.1_ASM90153838v1_protein.faa')

50S ribosomal protein L1 [
>WP_010040252.1 50S ribosomal protein L1 [Gemmata obscuriglobus]
 MANENEPAVAGAEVALAAPATPPADSPAPAAVEAVEKKVVEAELPQAKKNKKPGVGPRRGKKLRNHLNNVKKKLEDAGSIPVKQAVAELKKLKRAKFDETVELHINLGIDPTQSDQMVRGAIPLPHGIGKSVRVAVFCSGDDVAKAKAAGADVVGGADLVEKVQKENFLEFDVVLAVQAMMGQVSRLGKVLGPRGLMPTPKAGTVVPATGDLAQVVREFKAGKVEYRTDKTGQIHAGVGKLSFDEQKLVENITTFVEQIRSVKPSGVKGNFINGVVLSATMSPGIRLSI
1
50S ribosomal protein L2 [
>WP_010043965.1 50S ribosomal protein L2 [Gemmata obscuriglobus]
 MGVKQYKPTSAGRRAGMVSDFADCTTPRANSPEKKLLKPKKKKGGRNNQGIVTSRFRGGGHKQRYRVIDFKRKRDDVAATVISVEYDPNRTSRIALIEYPRDEKHDFSRAYILAPNGLKAGDKVISGEGDAVEPKPGNCMPLWKVPLGMTVHNVELVPGKGGQICRSAGCGAVLTAREKEWAQLTMPSGEIRRVSSKCRATIGVVSNAEHMNISIGKAGRMRWKGRKPHNRGTTMNPTDHPMGGGEGRTAGGRNPCSKTGVPAKGGKTRHKRKPGGKAIIRRRPAGRFQNTA
1
50S ribosomal protein L3 [
>WP_010050599.1 50S ribosomal protein L3 [Gemmata obscuriglobus]
 MALGLIGYKVGMTQVFTPEGKAEPVTVLQLGPCPVLQIKYPSAEGGAAKDGYTAIQLGFKDKKRKSATRPEQGHVAAGLKSKRKDARQKAGVVLPPKADCEPQKVIREFRIDLGGAFVQTGTIKAGPAEASFKV

In [43]:
### BEGIN ROUTINE ###

In [44]:
# S2
# >WP_197905127.1 chosen for Gemmata_obscuriglobus_UQM_2246

#>WP_197905127.1 30S ribosomal protein S2 [Gemmata obscuriglobus]
# MAPARENPNRVDLNSHIASMMTEAGKLDPALAPDTADILTHSKKMISTLSRELLKIRRNLMGIRDMIKPPDALVIVGPNKEHIAVKEAKRMGVTTIALIDTDSDPDPVDLPIPGNDDSIRSIEVVLAKLADACLEGRAALPPEQQGLPKRAPQPPRQQPQQPPTDTNPAPAGAV
    
    
# >Gemmata_obscuriglobus_UQM_2246
# MALVDVKVLLEAGVHYGHRASRWNPKMRPYIYGKRNLIHIIDLRETVRGLLRAYRYLSQVVGKGGLVMFVGTKRQAKEIVEREAQRCGMPFVSERWLGGTLTNYRTIRNRLKRLQELEAMWLPQGENPNRVDLNSHIASMMTEAGKLDPALAPDTADILTHSKKMISTLSRELLKIRRNLMGIRDMIKPPDALVIVGPNKEHIAVKEAKRMGVTTIALIDTDSDPDPVDLPIPGNDDSIRSIEVVLAKLADACLEGRAALPPEQQGLPKRAPQPPRQQPQQPPTDTNPAPAGAV
# >Gemmata_obscuriglobus_UQM_2246
# MAPARENPNRVDLNSHIASMMTEAGKLDPALAPDTADILTHSKKMISTLSRELLKIRRNLMGIRDMIKPPDALVIVGPNKEHIAVKEAKRMGVTTIALIDTDSDPDPVDLPIPGNDDSIRSIEVVLAKLADACLEGRAALPPEQQGLPKRAPQPPRQQPQQPPTDTNPAPAGAV
# >Gemmata_obscuriglobus_UQM_2246
# MRGLLRAYRYLSQVVGKGGLVMFVGTKRQAKEIVEREAQRCGMPFVSERWLGGTLTNYRTIRNRLKRLQELEAMWLPQGRTRTAST

In [45]:
# S4
# >WP_029601282.1 chosen for Gemmata

# >WP_029601282.1 30S ribosomal protein S4 [Gemmata obscuriglobus]
# MARNTDPVCKMCRRDQMKLYLKGDRCFGPKCPIDKQKPPPGMHGLRRSKTSEYGIRLREKQKLKRFYGVLEAQFRRYYGLATKAVGNTGEQLLSILERRLDNVVHRLGFAVNRTSARQMVSHGHILVNGVKCDIASMLVKPGDVIKVKTRDSSLKMARDTLAKGTVRIPDFLELTGTQDQPEGRMNRTPTRQDVDERITDIREQLIIEIATR

#30S ribosomal protein S4 [
#>WP_010049417.1 30S ribosomal protein S4 [Gemmata obscuriglobus]
# MSRHSQPKGKVNRALGMVIYEKRGAVKALERREYRPGMHGQRKGKVSEYGLALREKKKIKHYYGLHERQLVRFYQLATKSPENTGTELLVLCERRLDNVVRRAGFARTRDEARQGVNHGHFQVNGRKVDVASYLVKAGDVVAVKARPNLQKKYRERLTEGLHSADWLVSDAEALSFTVTRLPADGDVSLPVDIQRVIEILSR

#>WP_029601282.1 30S ribosomal protein S4 [Gemmata obscuriglobus]
# MARNTDPVCKMCRRDQMKLYLKGDRCFGPKCPIDKQKPPPGMHGLRRSKTSEYGIRLREKQKLKRFYGVLEAQFRRYYGLATKAVGNTGEQLLSILERRLDNVVHRLGFAVNRTSARQMVSHGHILVNGVKCDIASMLVKPGDVIKVKTRDSSLKMARDTLAKGTVRIPDFLELTGTQDQPEGRMNRTPTRQDVDERITDIREQLIIEIATR

In [46]:
# L10
# >WP_010040254.1 chosen for Gemmata
# >WP_010040254.1

#50S ribosomal protein L10 [
#>WP_010040254.1 50S ribosomal protein L10 [Gemmata obscuriglobus]
# MSKKIKELELNSLRKTFQGVRDFVLLEPLKLDSAADYTLRKSLREKKVRVKMVKNSFVRKVFSENGMKVETGSGPTLLCWGADSAKALGTAVEAALKLLRPDLKVPEKVKEKTGIADGEIMTLAQLAKVPTKQEAIGEVVAAVLAPGAALAGALMAPGAELAGILKAIEEKQSEGAAAPAAAS
#>WP_010047364.1 50S ribosomal protein L10 [Gemmata obscuriglobus]
# MSKKIKELELNSLRKTFQGVRDFVLLEPLKLDSAADYTLRKSLREKKVRVKMVKNSFVRKVFSENGMTVETGSGPTLLCWGADSAKSLGTAVEAALKLIRPDLKVPEKVKEKVGIADGETMTLAQLAKVPTKQEAIGEVLAAVLAPGAALAGALTAPGADLAGILKAVQGVRAQRTPAAVAVGATDSAWASEPAPSVPGPFVRPAAGAGARLPLPSVPRPPNNGVEPGAAPVRSSVRNAEPDPEVALRALERWLPLLHGIGAGRAPERVRTIWGEHVRNELFRSPPAVAWLDQCLAARPSFAPQRAAALEALRRVRAEVRDELTGLGEQLAPQAEGTSPPNGGSEMDQFILVLEIGARDGGPLTVGHEATLRAVVAKADVANTEDHYNGHYNWVPAGPGEVVAVRVFATGSRVLPAERYLSTPPQPGQSVEFTFRPRAAGRQYLHLALVDRHDSDVYTASQYFTVEPAAAVG


In [47]:
# S5 >WP_022478882.1 chosen for Coraliomargarita sp. CAG:312
#>WP_022478882.1
#
#30S ribosomal protein S5 [
#>WP_022478881.1 30S ribosomal protein S5 [Coraliomargarita sp. CAG:312]
# MSREFKNKQAIAEKDAGPELIEKVVHVNRCAKVVKGGRRXXXKAAAVLLFRRL
#>WP_022478882.1 30S ribosomal protein S5 [Coraliomargarita sp. CAG:312]
# MGDGAGKVGLGFGKAKEVPDAIKKATDRAKKVMKTVSLKDNTIPHEVLGEYDGGIVMLRPASPGTGVIAGGGVRAVLEAVGVKDVLSKSLGSNNDMAMVNATLNALYQLRTADAINAVRGK

In [48]:
#

In [49]:
### AUTO create files
#All in one -- one shot, iterate over genome list, grab fasta, rewrite, print to screen
#Manually build files.
#Look for all Large subunits in all Taxa, print those that don't have 15.
#There's got to be a better way to do this!!!

def buildFasta(protein,filename_protein):
    sys.stdout = open(protein+".fasta", "w") #Output file
    for index,row in taxa_df.iterrows():
        for header, seq in fasta_iter(index):
            if header.find(protein) != -1:
                new_header = taxa_df.loc[index][1]
                new_header =  new_header.replace(' ', '_')
                print('>'+new_header+'\n'+seq) #Turn this on to get specific.
            else:
                continue
    sys.stdout.close()

In [50]:
def buildFasta2(protein,filename_protein):
    sys.stdout = open(filename_protein+".fasta", "w") #Output file
    for index,row in taxa_df.iterrows():
        for header, seq in fasta_iter(index):
            if header.find(protein) != -1:
                new_header = taxa_df.loc[index][1]
                new_header =  new_header.replace(' ', '_')
                print('>'+new_header+'\n'+seq) #Turn this on to get specific.
            else:
                continue
    sys.stdout.close()

In [51]:
!pwd

/nobackup1c/users/payette/PVCtree/sequences


In [ ]:
#Example build one fasta
buildFasta2('50S ribosomal protein L1 [','L1')

In [ ]:
buildFasta2('50S ribosomal protein L2 [','L2')

In [ ]:
buildFasta2('50S ribosomal protein L3 [','L3')

In [ ]:
buildFasta2('50S ribosomal protein L4 [','L4')

In [ ]:
buildFasta2('50S ribosomal protein L5 [','L5')

In [ ]:
buildFasta2('50S ribosomal protein L6 [','L6')

In [ ]:
buildFasta2('50S ribosomal protein L10 [','L10')

In [ ]:
buildFasta2('50S ribosomal protein L13 [','L13')

In [ ]:
buildFasta2('50S ribosomal protein L14 [','L14')

In [ ]:
buildFasta2('50S ribosomal protein L15 [','L15')

In [ ]:
buildFasta2('50S ribosomal protein L18 [','L18')

In [ ]:
buildFasta2('50S ribosomal protein L22 [','L22')

In [ ]:
buildFasta2('50S ribosomal protein L23 [','L23')

In [ ]:
buildFasta2('50S ribosomal protein L24 [','L24')

In [ ]:
buildFasta2('50S ribosomal protein L29 [','L29')

In [ ]:
buildFasta2('30S ribosomal protein S2 [','S2')

In [ ]:
buildFasta2('30S ribosomal protein S3 [','S3')

In [ ]:
buildFasta2('30S ribosomal protein S4 [','S4')

In [ ]:
buildFasta2('30S ribosomal protein S5 [','S5')

In [ ]:
buildFasta2('30S ribosomal protein S7 [','S7')

In [ ]:
buildFasta2('30S ribosomal protein S8 [','S8')

In [ ]:
buildFasta2('30S ribosomal protein S9 [','S9')

In [ ]:
buildFasta2('30S ribosomal protein S10 [','S10')

In [ ]:
buildFasta2('30S ribosomal protein S11 [','S11')

In [ ]:
buildFasta2('30S ribosomal protein S12 [','S12')

In [ ]:
buildFasta2('30S ribosomal protein S13 [','S13')

In [ ]:
buildFasta2('30S ribosomal protein S14 [','S14')

In [ ]:
buildFasta2('30S ribosomal protein S15 [','S15')

In [ ]:
buildFasta2('30S ribosomal protein S17 [','S17')

In [ ]:
buildFasta2('30S ribosomal protein S19 [','S19')

In [52]:
# Warning these loops take a long time and you may need to restart your python kernel after they complete.

In [53]:
#
#for each in large_subunits:
#    buildFasta(each)

In [54]:
#
#for each in small_subunits:
#        buildFasta(each) 

In [55]:
# May need to restart kernel!

In [56]:
#MANUAL RENAMING OF FILES need to occur for the code below to work

In [57]:
large_subunits,small_subunits

(['50S ribosomal protein L1 [',
  '50S ribosomal protein L2 [',
  '50S ribosomal protein L3 [',
  '50S ribosomal protein L4 [',
  '50S ribosomal protein L5 [',
  '50S ribosomal protein L6 [',
  '50S ribosomal protein L10 [',
  '50S ribosomal protein L13 [',
  '50S ribosomal protein L14 [',
  '50S ribosomal protein L15 [',
  '50S ribosomal protein L18 [',
  '50S ribosomal protein L22 [',
  '50S ribosomal protein L23 [',
  '50S ribosomal protein L24 [',
  '50S ribosomal protein L29 ['],
 ['30S ribosomal protein S2 [',
  '30S ribosomal protein S3 [',
  '30S ribosomal protein S4 [',
  '30S ribosomal protein S5 [',
  '30S ribosomal protein S7 [',
  '30S ribosomal protein S8 [',
  '30S ribosomal protein S9 [',
  '30S ribosomal protein S10 [',
  '30S ribosomal protein S11 [',
  '30S ribosomal protein S12 [',
  '30S ribosomal protein S13 [',
  '30S ribosomal protein S14 [',
  '30S ribosomal protein S15 [',
  '30S ribosomal protein S17 [',
  '30S ribosomal protein S19 ['])

In [58]:
#TO DO MAKE A PROTEIN DF so I can call name and string search easily.
proteins = ['L1','L2','L3','L4','L5','L6',
            'L10','L13','L14','L15','L18','L22',
            'L23','L24','L29','S2','S3','S4',
            'S5','S7','S8','S9','S10','S11','S12',
            'S13','S14','S15','S17','S19']

In [59]:
def runAlign(protein):
    aln_output = open('/home/payette/work2/PVCtree/sequences/'+protein+'.aln', 'w')
    print(
        subprocess.call(['/home/payette/mafft/bin/mafft', '--reorder', '--auto',
                         '/home/payette/work2/PVCtree/sequences/'+protein+'.fasta'],
                    stdout=aln_output))
    aln_output.close()

In [60]:
# Open an existing fasta and manipulate 
def writeFasta(protein):
    sys.stdout = open(protein+".fasta", "w") #Output file
    fasta_nam = open(protein)                #Input txt file with protein
    fasta_s = (x[1] for x in groupby(fasta_nam, lambda line: line[0] == ">"))
    for header in fasta_s:                          
        headerStr = header.__next__()
        seq = "".join(s.strip() for s in fasta_s.__next__())
        headerStr = headerStr.replace(' ', '_')
        headerStr = re.search('(?<=\[).*?(?=\])',headerStr).group()
        print('>'+headerStr+'\n'+seq)
    sys.stdout.close()
    fasta_nam.close()

In [61]:
### RUN ALIGNMENTS ###

In [62]:
#

In [ ]:
runAlign('L1')

In [ ]:
runAlign('L2')

In [ ]:
runAlign('L3')

In [ ]:
runAlign('L4')

In [ ]:
runAlign('L5')

In [ ]:
runAlign('L6')

In [ ]:
runAlign('L10')

In [ ]:
runAlign('L13')

In [ ]:
runAlign('L14')

In [ ]:
runAlign('L15')

In [ ]:
runAlign('L18')

In [ ]:
runAlign('L22')

In [ ]:
runAlign('L23')

In [ ]:
runAlign('L24')

In [ ]:
runAlign('L29')

In [ ]:
runAlign('S2')

In [ ]:
runAlign('S3')

In [ ]:
runAlign('S4')

In [ ]:
runAlign('S5')

In [ ]:
runAlign('S7')

In [ ]:
runAlign('S8')

In [ ]:
runAlign('S9')

In [ ]:
runAlign('S10')

In [ ]:
runAlign('S11')

In [ ]:
runAlign('S12')

In [ ]:
runAlign('S13')

In [ ]:
runAlign('S14')

In [ ]:
runAlign('S15')

In [ ]:
runAlign('S17')

In [ ]:
runAlign('S19')

In [63]:
### INDIVIDUAL ALIGNMENT FINISHED! -- can run IQtree2 directly. 

In [64]:
from Bio import SeqIO
import sys
import re
import os

In [65]:
os.chdir('/home/payette/work/PVCtree/sequences')
!pwd

/nobackup1c/users/payette/PVCtree/sequences


In [66]:
!ls -lR *.aln | wc -l

30


In [67]:
from Bio import AlignIO, SeqIO, Align, Alphabet
import pandas as pd
import os, re, sys
from copy import deepcopy

aln_extension = '.aln'

In [68]:
!pwd

/nobackup1c/users/payette/PVCtree/sequences


In [69]:
aln_alphabet = Alphabet.Gapped(Alphabet.IUPAC.ambiguous_dna)
genomes    = {}
for aln in os.listdir('.'):
    #
    # check if files have the desired extension
    if not aln.endswith(aln_extension):
        continue
    alignment    = AlignIO.read(aln, 'fasta')
    genomes[aln] = set()
    print(alignment)
    for entry in alignment:
        genome = entry.name
        #print(genome)
        if genome in genomes[aln]:
            #print ('\t**Error, duplicated genome in %s: %s' %(aln, genome))
            sys.exit('\t**Error, duplicated genome in %s: %s' %(aln, genome))
            break
        genomes[aln].add(genome)

SingleLetterAlphabet() alignment with 104 rows and 326 columns
M----ARYVGPQCRLCRREGEKLFLKGIRCDTVKCA--------...--- Candidatus_Brocadia_sp.
M----GRYIGSVCKRCRREGVKLFLKGERCNSGKCS--------...--- Candidatus_Brocadiae_bacterium
M----GRYTGPVCRLCRREGIKLYLKGTRCDSPKCA--------...--- Isosphaera_pallida_ATCC_43644
M----GRHTGPVCRLCRREGMKLFLKGSRCDTPKCA--------...--- Paludisphaera_borealis
M----GRHIGPVCRLCRREGLKLFLKGTRCDTPKCA--------...--- Singulisphaera_acidiphila_DSM_18658
M----GRHIGPVCRLCRREGIKLFLKGTRCDSPKCA--------...--- Tautonia_marina
M----ARYTGPVCRLCRREGMKLFLKGTRCDTDKCA--------...--- Thermogutta_terrifontis
M----ARYTGPACRLCRRDGLKLFLKGTRCDTSKCA--------...--- Posidoniimonas_polymericola
M----ARYTGPVCRLCRRDGMKLFLKGFRCDSTKCA--------...--- Bythopirellula_goksoyri
M----ARNTDPVCKMCRRDQMKLYLKGDRCFGPKCP--------...--- Gemmata_obscuriglobus_UQM_2246
M----GRYLGPSCKQCRREGVKLFLKGMKCETAKCQ--------...--- Sedimentisphaerales_bacterium
M----SKYLGSVCKLCRREGEKLFLKGQRCDSPKCS--------...--- Candidatus_Poribacteria_bact

In [70]:
genome_union  = set.union(*genomes.values())
missing_genes = {} # just to keep track of the number of missing marker genes in each genome
concatenation = {}
for genome in genome_union:
    missing_genes[genome]             = 0
    concatenation[genome]             = Align.SeqRecord( Align.Seq('', aln_alphabet) )
    concatenation[genome].name        = genome
    concatenation[genome].id          = genome
    concatenation[genome].description = genome

In [71]:
#
# fill the handles with the marker sequences from each genome
total_genes      = 30 # keep track of the number of genes added to the concatenation
current_position = 1
partitions       = open('PVCtree_concatenated_partitions', 'w')
for aln in os.listdir('.'):
    if not aln.endswith(aln_extension):
        continue
    
    print(aln.replace(aln_extension, ''))
    tmp_aln      = AlignIO.read(aln, 'fasta' )
    aln_length   = tmp_aln.get_alignment_length() # get the expected size of the alignment so you can 
                                                  #   compare if all have the same size
    total_genes += aln_length

    for entry in tmp_aln:
        # if this alignment has a different size from the rest, something is reaaaaaly wrong!
        if len(entry) != aln_length:
            sys.exit('\t**Error, block "%s" has a different length than the rest of the MSA: %s' % 
                     (entry.name, aln))

        genome = entry.name
        concatenation[genome] += deepcopy(entry.seq)

    partitions.write('LG, %s = %i-%i\n' %
                     (aln.replace(aln_extension, ''), current_position, current_position+aln_length-1) )
    current_position += aln_length

    #
    # add gaps for those genomes missing this gene (same size as the expected alignment)
    for genome in genome_union.difference(genomes[aln]):
        concatenation[genome] += Align.Seq( '-' * aln_length, aln_alphabet )
        missing_genes[genome] += aln_length
partitions.close()

S4
S12
S5
L22
S15
S9
L4
L2
S3
S17
S2
L18
S8
S14
S11
L10
L6
L15
L14
L3
L29
L13
S7
L5
S10
S19
S13
L24
L1
L23


In [72]:
# Don't use!
# remove genomes missing more than 20% of the marker genes
counter = 0
for genome, num_missing_genes in missing_genes.items():
    if num_missing_genes/total_genes > 0.8:
        print('\t\t**%s: missing %.2f%% from concatenated alignment!' %(genome, (num_missing_genes/total_genes)*100))
        counter += 1

print('%i genomes missing more than 10%%' %counter)

0 genomes missing more than 10%


In [73]:
AlignIO.write(Align.MultipleSeqAlignment(concatenation.values() ), 'PVCtree.aln', 'fasta')

1

In [74]:
####
# Might have to manually adjust .aln file for duplicate names in Buchnera_a taxa!

In [80]:
from Bio import SeqIO
import sys
import re
import os
#!conda install -c bioconda iqtree

In [81]:
aln_name   = 'PVCtree.aln'
fasta      = open(aln_name).read()
taxa_names = re.findall('^>(\S+?)[\s|]', fasta, re.M)
for count, name in enumerate(taxa_names):
    fasta = re.sub('^>%s' % name, '>taxa_%i_ ' % count, fasta, flags=re.M)

In [82]:
out = open('PVCtree.fas', 'w')
out.write(fasta)
out.close()

In [84]:
taxa_names

['Roseiconus_lacunae',
 'Maioricimonas_rarisocia',
 'Chlamydiifrater_phoenicopteri',
 'Geminisphaera_colitermitum_TAV2',
 'Planctopirus_limnophila_DSM_3776',
 'Methylacidimicrobium_tartarophylax',
 'Methylacidiphilum_fumariolicum_SolV',
 'Opitutus_terrae_PB90-1',
 'Roseimaritima_sp._JC651',
 'Lentisphaerae_bacterium_RIFOXYA12_64_32',
 'Calycomorphotria_hydatis',
 'Lentisphaerae_bacterium',
 'Rosistilla_ulvae',
 'Simkania_negevensis_Z',
 'Candidatus_Clavichlamydia_salmonicola',
 'Victivallis_vadensis',
 'Symmachiella_macrocystis',
 'Methylacidiphilum_kamchatkense_Kam1',
 'Akkermansia_muciniphila',
 'Coraliomargarita_akajimensis_DSM_45221',
 'Lentisphaera_araneosa_HTCC2155',
 'Candidatus_Brocadia_sp.',
 'Anatilimnocola_aggregata',
 'Waddlia_chondrophila_WSU_86-1044',
 'Thalassoglobus_polymorphus',
 'Stieleria_varia',
 'Symmachiella_dynata',
 'Rubinisphaera_brasiliensis_DSM_5305',
 'Terrimicrobium_sacchariphilum',
 'Humisphaera_borealis',
 'Candidatus_Neptunochlamydia_vexilliferae',
 'Can

In [85]:
SeqIO.convert('PVCtree.fas', 'fasta', 'PVCtree.phy', 'phylip')

ValueError: Repeated name 'taxa_11_' (originally 'taxa_11_'), possibly due to truncation

In [86]:
SeqIO.convert('PVCtree.fas', 'fasta', 'PVCtree.phy', 'phylip')
phylip = open('PVCtree.phy').read()
for count, name in enumerate(taxa_names):
    phylip = re.sub('^taxa_%i_\s+' % count, '%s  ' % name, phylip, flags=re.M)

ValueError: Repeated name 'taxa_11_' (originally 'taxa_11_'), possibly due to truncation

In [ ]:
out = open('%s.phy' % aln_name, 'w')
out.write(phylip)
out.close()

In [ ]:
#!/nobackup1c/users/payette/iqtree-2.1.2-Linux/bin/iqtree2 -nt AUTO -ntmax 10 -s PVCtree.aln.phy -m MFP -mrate E,I,G4,I+G4,R4 -madd LG4X,C20 -keep-ident -safe -mem 15G -bb 1000 -alrt 1000

In [ ]:
!pwd

In [ ]:
# END ############

In [104]:
import subprocess

In [105]:
### Treemmer addendum routine
# Filter out the original fastas by using the BuildFasta script and Treemmer output

In [106]:
PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50

NameError: name 'PVCtree_2_1_22' is not defined

In [115]:
os.chdir('/home/payette/pool001/Treemmer/Treemmer-master/PVCtree/')

In [ ]:
S4
S12
S5
L22
S15
S9
L4
L2
S3
S17
S2
L18
S8
S14
S11
L10
L6
L15
L14
L3
L29
L13
S7
L5
S10
S19
S13
L24
L1
L23

In [118]:
!python3 BuildFasta.py S4.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S4-T.fasta

In [119]:
!python3 BuildFasta.py S12.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S12-T.fasta

In [120]:
!python3 BuildFasta.py S5.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S5-T.fasta

In [121]:
!python3 BuildFasta.py L22.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 L22-T.fasta

In [122]:
!python3 BuildFasta.py S15.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S15-T.fasta

In [123]:
!python3 BuildFasta.py S9.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S9-T.fasta

In [124]:
!python3 BuildFasta.py L4.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 L4-T.fasta

In [125]:
!python3 BuildFasta.py L2.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 L2-T.fasta

In [126]:
!python3 BuildFasta.py S3.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S3-T.fasta

In [127]:
!python3 BuildFasta.py S17.fasta PVCtree_2_1_22-Copy1.treefile_trimmed_list_X_50 S17-T.fasta

In [ ]:
proteins